# Unsupervised Anomaly Detection in Time Series of Bank Transactions

## 1. Data Cleaning & Exploratory Data Analysis

In [ ]:
# Data and Stats Packages
import pandas as pd
import numpy as np
import re
import datetime as dt

# Visualization Packages
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# Anomaly Detection Packages
from sklearn import preprocessing
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.covariance import EllipticEnvelope
from statsmodels.tsa.seasonal import seasonal_decompose


import warnings
warnings.filterwarnings("ignore")

In [ ]:
bank = pd.read_excel('Sample_bank_transaction_data.xlsx')
bank_raw = bank.copy()

In [ ]:
bank.head()

In [ ]:
bank[bank['BALANCE AMT']<0].shape

In [ ]:
bank[bank['WITHDRAWAL AMT']<0].shape

In [ ]:
bank[bank['DEPOSIT AMT']<0].shape

In [ ]:
bank.shape

In [ ]:
bank.describe()

In [ ]:
bank.rename(columns={'Account No': 'account_no', 'DATE':'date', 'TRANSACTION DETAILS':'details', 'CHQ.NO.':'chq_no', 'VALUE DATE':'value_date',
       'WITHDRAWAL AMT':'withdrawal_amt', 'DEPOSIT AMT':'deposit_amt', 'BALANCE AMT':'balance_amt'}, inplace=True)
bank.head(3)

We have a time series data without anomaly labeling. Let's take a closer look at the features.

In [ ]:
def first_look(df, col):
    """
    Enables a quick search for characteristics of a column in a data frame.
    
    df: Data Frame
    col: Column of interest
    
    """
    val = pd.DataFrame(df[col].value_counts(dropna=False))
    val = val.assign(percent=(df[col].value_counts(dropna=False, normalize=True).values)*100)
    info = pd.DataFrame({'Column_Name': [col],'Null_Rows': [df[col].isna().sum()], 'Column_Type': [df[col].dtypes]}, index=['column_info'])
    return info, val

### i. account_no

In [ ]:
info , val = first_look(bank, bank.columns[0])

In [ ]:
info

In [ ]:
val

Since **account_no** is a unique indicator of customers (in our case we have 10 customers who have rather unbalanced transaction frequencies), we should keep the data type as string and for future investigation purposes we will remove the **'** (last character).

In [ ]:
bank['account_no'] = bank.account_no.str.extract(r'(\d+)\'')

In [ ]:
plt.figure(figsize=(10,6))
plt.xticks(rotation=45)
fig = sns.countplot(data=bank, x='account_no', order=bank.account_no.value_counts().index)
fig2 = fig.twinx()

fig.yaxis.set_label_position('left')
fig2.yaxis.set_label_position('right')

fig2.set_ylabel('% of total')

for p in fig.patches:
    x=p.get_bbox().get_points()[:,0]
    y=p.get_bbox().get_points()[1,1]
    fig.annotate('{:.2f}%'.format(100.*y/len(bank)), (x.mean(), y), 
            ha='center', va='bottom') # set the alignment of the text
fig2.grid(None)
fig2.set_ylim(0,100)
fig2.grid()
plt.show()

### ii. date

In [ ]:
info , val = first_look(bank, bank.columns[1])

In [ ]:
info

In [ ]:
bank.date.apply(lambda x: x.year).value_counts()

In [ ]:
((bank.date).dt.dayofweek).value_counts()

In [ ]:
conditions = [((bank.date).dt.dayofweek  > 4), ((bank.date).dt.dayofweek  < 5)]
values = [1, 0]
bank['weekend_date'] = np.select(conditions, values)

### iii. details

In [ ]:
info , val = first_look(bank, bank.columns[2])

In [ ]:
info

In [ ]:
bank[bank.details.isna()].account_no.value_counts()

In [ ]:
bank[bank.details.isna()].date.apply(lambda x: x.year).value_counts()

In [ ]:
bank.details.sample(10)

This function has the potential to provide information about anomalies. It could help us find new features. For starters, let's define a column that indicates whether a transaction has no proper details value (Null or nonsense values).

In [ ]:
conditions = [(bank.details.isna())]
values = [1]
bank['details_null_penalty'] = np.select(conditions, values)

In [ ]:
conditions = [(bank.details.str.extract(r'(\w+)')[0].isna())]
values = [1]
bank['details_numbers_penalty'] = np.select(conditions, values)

In [ ]:
bank.details_null_penalty.value_counts()

In [ ]:
bank.details_numbers_penalty.value_counts()

This penalty is not a red flag by itself, but can be an additional checkpoint for the anomaly detection results. Let's leave the **details** column at this point and move on to the next feature.

### iv. chq_no

In [ ]:
info , val = first_look(bank, bank.columns[3])

In [ ]:
info

In [ ]:
val

In [ ]:
bank[bank.chq_no<10]

In [ ]:
bank[~bank.chq_no.isna()].sample(10)

In [ ]:
bank[bank.chq_no<100000].tail()

In [ ]:
bank['cheque_penalty'] = np.where(bank.chq_no<11, 1, 0)

In [ ]:
bank.cheque_penalty.value_counts()

Again, this penalty is not a red flag in itself, but on the other hand, given the incorrect check numbers, it can be an additional check point for the anomaly detection results. Let's leave the **chq_no** column at this point and move on to the next feature.

### v. value_date

In [ ]:
info , val = first_look(bank, bank.columns[4])

In [ ]:
info

In [ ]:
bank['day_diff'] = ((bank.date - bank.value_date) / np.timedelta64(1,'D')).astype(int)

In [ ]:
bank.day_diff.value_counts()

In [ ]:
bank[(bank.day_diff < 0) | (bank.day_diff > 5)]

As one could obviously see, we have obtained an important feature that we will look at more closely in the anomaly analysis. Let's have a quick look whether we have weekend value_dates.

In [ ]:
((bank.value_date).dt.dayofweek  > 4).value_counts()

In [ ]:
conditions = [((bank.value_date).dt.dayofweek  > 4), ((bank.value_date).dt.dayofweek  < 5)]
values = [1, 0]
bank['weekend_value_date'] = np.select(conditions, values)

### vi. withdrawal_amt

In [ ]:
info , val = first_look(bank, bank.columns[5])

In [ ]:
info

In [ ]:
bank.withdrawal_amt.describe()

In [ ]:
bank[~bank.withdrawal_amt.isna()].count()

In [ ]:
bank[bank.withdrawal_amt.isna()].count()

In [ ]:
pd.DataFrame(bank.groupby('account_no').withdrawal_amt.count()).sort_values('withdrawal_amt', ascending=False)

In [ ]:
conditions = [((bank.balance_amt+bank.withdrawal_amt)==0), ((bank.balance_amt+bank.withdrawal_amt)!=0)]
values = [0, 100*(bank.withdrawal_amt/(bank.balance_amt+bank.withdrawal_amt))]
bank['withdrawal_over_balance'] = np.select(conditions, values)

In [ ]:
bank['withdrawal_amt'] = bank.withdrawal_amt.fillna(0)

In [ ]:
bank['withdrawal_over_balance'] = bank.withdrawal_over_balance.fillna(0)

In [ ]:
bank[bank.withdrawal_amt!=0].head()

### vii. deposit_amt

In [ ]:
info , val = first_look(bank, bank.columns[6])

In [ ]:
info

In [ ]:
bank.deposit_amt.describe()

In [ ]:
bank[~bank.deposit_amt.isna()].count()

In [ ]:
bank[bank.deposit_amt.isna()].count()

In [ ]:
pd.DataFrame(bank.groupby('account_no').deposit_amt.count()).sort_values('deposit_amt', ascending=False)

In [ ]:
bank[bank.deposit_amt.isna() & bank.withdrawal_amt.isna()].count()

In [ ]:
conditions = [((bank.balance_amt-bank.deposit_amt)==0), ((bank.balance_amt-bank.deposit_amt)!=0)]
values = [0, 100*(bank.deposit_amt/(bank.balance_amt-bank.deposit_amt))]
bank['deposit_over_balance'] = np.select(conditions, values)
bank['deposit_over_balance'] = bank.deposit_over_balance.fillna(0)

In [ ]:
bank['deposit_amt'] = bank.deposit_amt.fillna(0)

### viii. balance_amt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
info , val = first_look(bank, bank.columns[7])

In [ ]:
info

In [ ]:
bank.balance_amt.describe()

In [ ]:
bank[bank.balance_amt.isna()].count()

### ix. Dropping Empty Column

In [ ]:
info , val = first_look(bank, bank.columns[8])

In [ ]:
info

In [ ]:
val

In [ ]:
bank.drop(columns=['.'], inplace=True, axis=1)

In [ ]:
bank_detailed = bank.copy()

In [ ]:
bank = bank[['account_no', 'date', 'value_date', 'day_diff', 'weekend_date', 'weekend_value_date', 'withdrawal_amt', 'deposit_amt', 'balance_amt', 'withdrawal_over_balance', 'deposit_over_balance', 'details_null_penalty', 'details_numbers_penalty', 'cheque_penalty']]

In [ ]:
bank.head()

## 2. Data Visualization

In [ ]:
plt.figure(figsize=(18, 18))
sns.heatmap(bank.corr(), annot=True)
plt.yticks(rotation=0)
plt.show()

In [ ]:
features = ['day_diff', 'withdrawal_amt', 'deposit_amt', 'balance_amt', 'withdrawal_over_balance', 'deposit_over_balance']

In [ ]:
fig, axes = plt.subplots(2,3, sharex=False, figsize=(18, 12))
fig.suptitle('Outliers by Feature', color='r' )

for i in range(2):
    for j in range(3):
        sns.boxplot(ax=axes[i,j], x= features[3*i + j], data = bank)
        axes[i,j].set_title(f'{features[3*i + j]}')

plt.tight_layout()
plt.show()

In [ ]:
features = ['day_diff', 'withdrawal_amt', 'deposit_amt', 'balance_amt', 'withdrawal_over_balance', 'deposit_over_balance']
fig, axes = plt.subplots(3,2, sharex=False, figsize=(18, 18))
fig.suptitle('Lineplots by Account Number', color='r' )

for i in range(3):
    for j in range(2):
        sns.lineplot(ax=axes[i,j], y= features[2*i + j], x = 'date', data = bank, hue = 'account_no')
        axes[i,j].set_title(f'{features[2*i + j]}')

plt.tight_layout()
plt.show()

## 3. Anomaly Detection Whole Dataset (Account No. as a Feature)

Before getting started with the anomaly detection we need to convert categorical variable into dummy/indicator variables. 

In [ ]:
bank_premodel = bank.copy()

In [ ]:
bank = pd.get_dummies(bank, columns = ['account_no'])

In [ ]:
bank.columns

In [ ]:
bank.head(3)

In [ ]:
(bank.weekend_date + bank.weekend_value_date).value_counts()

### i. Isolation Forest Anomaly Detection

- Isolation Forest returns the anomaly score of each sample using the IsolationForest algorithm. 

- The IsolationForest ‘isolates’ observations by randomly selecting a feature and then randomly selecting a split value between the maximum and minimum values of the selected feature.

- Since recursive partitioning can be represented by a tree structure, the number of splittings required to isolate a sample is equivalent to the path length from the root node to the terminating node.

- This path length, averaged over a forest of such random trees, is a measure of normality and our decision function.

- Random partitioning produces noticeably shorter paths for anomalies. Hence, when a forest of random trees collectively produce shorter path lengths for particular samples, they are highly likely to be anomalies.

link : https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.IsolationForest.html

In [ ]:
from sklearn.preprocessing import StandardScaler
bank_iso = bank.drop(columns=['date', 'value_date'], axis=1) 
scaler = StandardScaler()
bank_iso_scaled = pd.DataFrame(scaler.fit_transform(bank_iso), columns=bank_iso.columns)

In [ ]:
bank_main = bank.copy()
outliers_fraction = .007
model = IsolationForest(contamination=outliers_fraction)
model.fit(bank_iso_scaled)
bank_main['anomaly_isolation_forest'] = pd.Series(model.predict(bank_iso_scaled)).apply(lambda x: 0 if x==1 else 1)

In [ ]:
bank_main['anomaly_isolation_forest'].value_counts()

In [ ]:
features = ['account_no_1196428', 'account_no_1196711', 'account_no_409000362497', 'account_no_409000405747', 'account_no_409000425051', \
            'account_no_409000438611', 'account_no_409000438620', 'account_no_409000493201', 'account_no_409000493210', 'account_no_409000611074']
fig, axes = plt.subplots(5,2, sharex=False, figsize=(18, 18))
fig.suptitle('Anomalies by Account Number', color='r' )

for i in range(5):
    for j in range(2):
        
        df = bank_main[bank_main[features[2*i + j]]==1]
        a= df.loc[df['anomaly_isolation_forest']==1, ['date', 'balance_amt']]

        axes[i,j].plot(df['date'], df['balance_amt'], color='blue', label='Normal')
        axes[i,j].scatter(a['date'], a['balance_amt'], color='red', label='Anomaly')
        axes[i,j].set_title(f'Anomaly in Balance {features[2*i + j]}')
        axes[i,j].set_xlabel('Date')
        axes[i,j].set_ylabel('Balance amount')
        axes[i,j].legend()

plt.tight_layout()
plt.show()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
labels = bank_main.anomaly_isolation_forest
colors = {0:'blue', 1:'red'}

fig = plt.figure(1, figsize=(8,8))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(bank_iso_scaled.iloc[:,3], bank_iso_scaled.iloc[:,4], bank_iso_scaled.iloc[:,5], c=labels.apply(lambda x:colors[x]))
ax.set_title(f'ISOLATION FOREST (outliers_fraction = {100*outliers_fraction}%)', fontsize=14)
ax.set_xlabel('withdrawal_amt')
ax.set_ylabel('deposit_amt')
ax.set_zlabel('balance_amt')

### ii. OneClassSVM Anomaly Detection

In [ ]:
bank_svm = bank.drop(columns=['date', 'value_date'], axis=1) 
scaler = StandardScaler()
bank_svm_scaled = pd.DataFrame(scaler.fit_transform(bank_svm), columns=bank_svm.columns)

In [ ]:
outliers_fraction = .005
model = OneClassSVM(nu=outliers_fraction, kernel='rbf', gamma=.01)
model.fit(bank_svm_scaled)
bank_main['anomaly_one_class_svm'] = pd.Series(model.predict(bank_svm_scaled)).apply(lambda x: 0 if x==1 else 1)

In [ ]:
bank_main['anomaly_one_class_svm'].value_counts()

In [ ]:
features = ['account_no_1196428', 'account_no_1196711', 'account_no_409000362497', 'account_no_409000405747', 'account_no_409000425051', \
            'account_no_409000438611', 'account_no_409000438620', 'account_no_409000493201', 'account_no_409000493210', 'account_no_409000611074']
fig, axes = plt.subplots(5,2, sharex=False, figsize=(18, 18))
fig.suptitle('Anomalies by Account Number', color='r' )

for i in range(5):
    for j in range(2):
        
        df = bank_main[bank_main[features[2*i + j]]==1]
        a= df.loc[df['anomaly_one_class_svm']==1, ['date', 'balance_amt']]

        axes[i,j].plot(df['date'], df['balance_amt'], color='blue', label='Normal')
        axes[i,j].scatter(a['date'], a['balance_amt'], color='red', label='Anomaly')
        axes[i,j].set_title(f'Anomaly in Balance {features[2*i + j]}')
        axes[i,j].set_xlabel('Date')
        axes[i,j].set_ylabel('Balance amount')
        axes[i,j].legend()

plt.tight_layout()
plt.show()

In [ ]:
labels = bank_main.anomaly_one_class_svm
colors = {0:'blue', 1:'red'}

fig = plt.figure(1, figsize=(8,8))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(bank_svm_scaled.iloc[:,3], bank_svm_scaled.iloc[:,4], bank_svm_scaled.iloc[:,5], c=labels.apply(lambda x:colors[x]))
ax.set_title(f'ONE CLASS SVM (outliers_fraction = {100*outliers_fraction}%)', fontsize=14)
ax.set_xlabel('withdrawal_amt')
ax.set_ylabel('deposit_amt')
ax.set_zlabel('balance_amt')

### iii. Gaussian Distribution Anomaly Detection (Elliptic Envelope)

In [ ]:
bank_gauss = bank.drop(columns=['date', 'value_date'], axis=1) 
scaler = StandardScaler()
bank_gauss_scaled = pd.DataFrame(scaler.fit_transform(bank_gauss), columns=bank_gauss.columns)

In [ ]:
outliers_fraction = .005
envelope =  EllipticEnvelope(contamination = outliers_fraction) 
X_train = bank_gauss_scaled.values
envelope.fit(X_train)
bank_gauss_scaled['deviation'] = envelope.decision_function(X_train)
bank_gauss_scaled['anomaly_gaussian'] = pd.Series(envelope.predict(X_train)).apply(lambda x: 0 if x==1 else 1)
bank_gauss_scaled = bank_gauss_scaled.fillna(0)
bank_main['anomaly_gaussian_elliptic_envelope'] = bank_gauss_scaled.anomaly_gaussian
bank_main['anomaly_gaussian_elliptic_envelope'].value_counts(dropna=False)

In [ ]:
labels = bank_main.anomaly_gaussian_elliptic_envelope
colors = {0.0:'blue', 1.0:'red'}

fig = plt.figure(1, figsize=(8,8))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(bank_gauss_scaled.iloc[:,3], bank_gauss_scaled.iloc[:,4], bank_gauss_scaled.iloc[:,5], c=labels.apply(lambda x:colors[x]))
ax.set_title(f'GAUSSIAN DISTRIBUTION (outliers_fraction = {100*outliers_fraction}%)', fontsize=14)
ax.set_xlabel('withdrawal_amt')
ax.set_ylabel('deposit_amt')
ax.set_zlabel('balance_amt')

In [ ]:
features = ['account_no_1196428', 'account_no_1196711', 'account_no_409000362497', 'account_no_409000405747', 'account_no_409000425051', \
            'account_no_409000438611', 'account_no_409000438620', 'account_no_409000493201', 'account_no_409000493210', 'account_no_409000611074']
fig, axes = plt.subplots(5,2, sharex=False, figsize=(18, 18))
fig.suptitle('Anomalies by Account Number', color='r' )

for i in range(5):
    for j in range(2):
        
        df = bank_main[bank_main[features[2*i + j]]==1]
        a= df.loc[df['anomaly_gaussian_elliptic_envelope']==1, ['date', 'balance_amt']]

        axes[i,j].plot(df['date'], df['balance_amt'], color='blue', label='Normal')
        axes[i,j].scatter(a['date'], a['balance_amt'], color='red', label='Anomaly')
        axes[i,j].set_title(f'Anomaly in Balance {features[2*i + j]}')
        axes[i,j].set_xlabel('Date')
        axes[i,j].set_ylabel('Balance amount')
        axes[i,j].legend()

plt.tight_layout()
plt.show()

### v. Clustering-Based Anomaly Detection

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from yellowbrick.cluster import KElbowVisualizer

In [ ]:
bank_clstr = bank.drop(columns=['date', 'value_date'], axis=1) 
scaler = StandardScaler()
bank_scaled = scaler.fit_transform(bank_clstr)

In [ ]:
# Instantiate the clustering model and visualizer
visualizer = KElbowVisualizer(KMeans(), k=(2,15))
plt.figure(figsize=(8,4)).patch.set_facecolor('xkcd:white')
visualizer.fit(bank_clstr)       # Fit the data to the visualizer
# visualizer.show()       # Finalize and render the figure

In [ ]:
kmeans = KMeans(n_clusters=5)
kmeans.fit(bank_clstr)

Let's visualise labeled clusters.

In [ ]:
labels = kmeans.labels_

fig = plt.figure(1, figsize=(8,8))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(bank_clstr.iloc[:,3], bank_clstr.iloc[:,4], bank_clstr.iloc[:,5], c=labels.astype(np.float), cmap='rainbow')
ax.set_title('K MEANS', fontsize=14)
ax.set_xlabel('withdrawal_amt')
ax.set_ylabel('deposit_amt')
ax.set_zlabel('balance_amt')

To overcome the curse of dimensionality, we need to figure out which components we want to keep.

In [ ]:
# Standardize/scale the dataset and apply PCA
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
# Extract the names of the numerical columns
x = bank_clstr
scaler = StandardScaler()
pca = PCA()
pipeline = make_pipeline(scaler, pca)
pipeline.fit(x)
# Plot the principal components against their inertia
features = range(pca.n_components_)
_ = plt.figure(figsize=(15, 5))
_ = plt.bar(features, pca.explained_variance_)
_ = plt.xlabel('PCA feature')
_ = plt.ylabel('Variance')
_ = plt.xticks(features)
_ = plt.title("Importance of the Principal Components based on inertia")
plt.show()


In [ ]:
bank_gen = bank_clstr[bank_clstr.columns[:11]]

# Extract the names of the numerical columns
x = bank_gen
scaler = StandardScaler()
pca = PCA()
pipeline = make_pipeline(scaler, pca)
pipeline.fit(x)
# Plot the principal components against their inertia
features = range(pca.n_components_)
_ = plt.figure(figsize=(15, 5))
_ = plt.bar(features, pca.explained_variance_)
_ = plt.xlabel('PCA feature')
_ = plt.ylabel('Variance')
_ = plt.xticks(features)
_ = plt.title("Importance of the Principal Components based on inertia")
plt.show()

It turns out that even a data frame without customer columns PCA does not help too much in our case.

In K-Means Clustering approach to anomaly detection we calculate the distance between each point and its nearest centroid and the biggest distances are considered as anomaly. We are going to take following steps:

- We assume a proportion of data as outliers (outliers_fraction: proportion of the outliers present in our data set) and calculate number_of_outliers using outliers_fraction.
- We set a threshold as the minimum distance of these outliers.
- After labeling (0:normal, 1:anomaly) anomalies we will visualize them.

In [ ]:
def getDistanceByPoint(data, model):
    """ 
    Function that calculates the distance between a point and centroid of a cluster, 
    returns the distances in pandas series
    """
    distance = []
    for i in range(0,len(data)):
        Xa = np.array(data.loc[i])
        Xb = model.cluster_centers_[model.labels_[i]-1]
        distance.append(np.linalg.norm(Xa-Xb))
    return pd.Series(distance, index=data.index)

bank_scaled = pd.DataFrame(bank_scaled, columns=bank_clstr.columns)
kmeans = KMeans(n_clusters=5)
kmeans.fit(bank_scaled)
labels = kmeans.predict(bank_scaled)
unique_elements, counts_elements = np.unique(labels, return_counts=True)
clusters = np.asarray((unique_elements, counts_elements))

# Assume that 0.5% of the entire data set are anomalies 
outliers_fraction = 0.005
# get the distance between each point and its nearest centroid. The biggest distances are considered as anomaly
distance = getDistanceByPoint(bank_scaled, kmeans)
# number of observations that equate to the 0.5% of the entire data set
number_of_outliers = int(outliers_fraction*len(distance))
# Take the minimum of the largest 0.5% of the distances as the threshold
threshold = distance.nlargest(number_of_outliers).min()
# anomaly1 contain the anomaly result of the above method Cluster (0:normal, 1:anomaly) 
bank_scaled['anomaly_kmeans'] = (distance >= threshold).astype(int)

In [ ]:
bank_scaled.anomaly_kmeans.value_counts()

In [ ]:
labels = bank_scaled.anomaly_kmeans
colors = {0:'blue', 1:'red'}

fig = plt.figure(1, figsize=(8,8))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(bank_scaled.iloc[:,3], bank_scaled.iloc[:,4], bank_scaled.iloc[:,5], c=labels.apply(lambda x:colors[x]))
ax.set_title(f'K MEANS ANOMOLIES (outliers_fraction = {100*outliers_fraction})%', fontsize=14)
ax.set_xlabel('withdrawal_amt')
ax.set_ylabel('deposit_amt')
ax.set_zlabel('balance_amt')

In [ ]:
bank_main['anomaly_kmeans'] = bank_scaled.anomaly_kmeans

In [ ]:
# bank_main = bank_clstr.copy()
# bank_main['date'], bank_main['value_date'], bank_main['anomaly_kmeans'] = bank_premodel.date, bank_premodel.value_date, bank_scaled.anomaly_kmeans

In [ ]:
features = ['account_no_1196428', 'account_no_1196711', 'account_no_409000362497', 'account_no_409000405747', 'account_no_409000425051', \
            'account_no_409000438611', 'account_no_409000438620', 'account_no_409000493201', 'account_no_409000493210', 'account_no_409000611074']
fig, axes = plt.subplots(5,2, sharex=False, figsize=(18, 18))
fig.suptitle('Anomalies by Account Number', color='r' )

for i in range(5):
    for j in range(2):
        
        df = bank_main[bank_main[features[2*i + j]]==1]
        a= df.loc[df['anomaly_kmeans']==1, ['date', 'balance_amt']]

        axes[i,j].plot(df['date'], df['balance_amt'], color='blue', label='Normal')
        axes[i,j].scatter(a['date'], a['balance_amt'], color='red', label='Anomaly')
        axes[i,j].set_title(f'Anomaly in Balance {features[2*i + j]}')
        axes[i,j].set_xlabel('Date')
        axes[i,j].set_ylabel('Balance amount')
        axes[i,j].legend()

plt.tight_layout()
plt.show()


Let's follow the same process, **but this time customer by customer**. To do this, we will split the bank data frame and run the clustering approach for each individual data frame.

In [ ]:
bank_main.columns

In [ ]:
bank_main['anomaly_sum_1'] = bank_main.anomaly_kmeans + bank_main.anomaly_isolation_forest + bank_main.anomaly_one_class_svm + bank_main.anomaly_gaussian_elliptic_envelope
bank_main.anomaly_sum_1.value_counts()

## 4. Anomaly Detection by Account Number

In this section, we will analyze each account number independently.

In [ ]:
accounts = ['1196428', '409000362497', '409000438620', '1196711', '409000493210', '409000438611', '409000611074', '409000493201', '409000425051', '409000405747']

acc1, acc2, acc3, acc4, acc5, acc6, acc7, acc8, acc9, acc10  = bank_premodel[bank_premodel.account_no== accounts[0]].drop('account_no', axis=1),\
    bank_premodel[bank_premodel.account_no == accounts[1]].drop('account_no', axis=1), bank_premodel[bank_premodel.account_no == accounts[2]].drop('account_no', axis=1),\
    bank_premodel[bank_premodel.account_no == accounts[3]].drop('account_no', axis=1), bank_premodel[bank_premodel.account_no == accounts[4]].drop('account_no', axis=1),\
    bank_premodel[bank_premodel.account_no == accounts[5]].drop('account_no', axis=1), bank_premodel[bank_premodel.account_no == accounts[6]].drop('account_no', axis=1),\
    bank_premodel[bank_premodel.account_no == accounts[7]].drop('account_no', axis=1), bank_premodel[bank_premodel.account_no == accounts[8]].drop('account_no', axis=1),\
    bank_premodel[bank_premodel.account_no == accounts[9]].drop('account_no', axis=1)
df_list = [acc1, acc2, acc3, acc4, acc5, acc6, acc7, acc8, acc9, acc10]

### i. Isolation Forest Anomaly Detection

#### &emsp;(I)  Isolation Forest account_no: 1196428

In [ ]:
acc1_iso = acc1.copy() 
scaler = StandardScaler()
acc1_iso_scaled = pd.DataFrame(scaler.fit_transform(acc1_iso[acc1.columns[2:]]), columns=acc1.columns[2:])
acc1_iso_scaled.set_index(acc1_iso.index, drop=True, inplace=True)
acc1_iso_scaled.head(3)

In [ ]:
outliers_fraction = .005
model = IsolationForest(contamination=outliers_fraction)
model.fit(acc1_iso_scaled)

In [ ]:
acc1_iso_scaled['anomaly_iso'] = pd.Series(model.predict(acc1_iso_scaled)).apply(lambda x: 0 if x==1 else 1).values
acc1_iso['anomaly_iso'] = acc1_iso_scaled['anomaly_iso']
acc1_iso_scaled['anomaly_iso'].value_counts()

In [ ]:
bank_main['iso_1'] = 0
bank_main.loc[acc1_iso_scaled.index,'iso_1'] = acc1_iso_scaled['anomaly_iso']
bank_main['iso_1'].value_counts()

In [ ]:
labels = acc1_iso_scaled.anomaly_iso
colors = {0:'blue', 1:'red'}

fig = plt.figure(1, figsize=(8,8))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(acc1_iso_scaled.iloc[:,3], acc1_iso_scaled.iloc[:,4], acc1_iso_scaled.iloc[:,5], c=labels.apply(lambda x:colors[x]))
ax.set_title(f'ISOLATION FOREST (account_no = {accounts[0]}, outliers_fraction = {100*outliers_fraction}%)', fontsize=14)
ax.set_xlabel('withdrawal_amt')
ax.set_ylabel('deposit_amt')
ax.set_zlabel('balance_amt')

In [ ]:
df = acc1_iso.copy()
a= df.loc[df['anomaly_iso']==1, ['date', 'balance_amt']]
plt.figure(figsize=(20,8))
plt.plot(df['date'], df['balance_amt'], color='blue', label='Normal')
plt.scatter(a['date'], a['balance_amt'], color='red', label='Anomaly')
plt.title(f'{accounts[0]} Isolation Forest Anomalies')
plt.xlabel('Date')
plt.ylabel('balance_amt')
plt.legend()
plt.show()

#### &emsp;(II)  Isolation Forest account_no: 409000362497

In [ ]:
acc2_iso = acc2.copy() 
scaler = StandardScaler()
acc2_iso_scaled = pd.DataFrame(scaler.fit_transform(acc2_iso[acc2.columns[2:]]), columns=acc2.columns[2:])
acc2_iso_scaled.set_index(acc2_iso.index, drop=True, inplace=True)
acc2_iso_scaled.head(3)

In [ ]:
outliers_fraction = .005
model = IsolationForest(contamination=outliers_fraction)
model.fit(acc2_iso_scaled)

In [ ]:
acc2_iso_scaled['anomaly_iso'] = pd.Series(model.predict(acc2_iso_scaled)).apply(lambda x: 0 if x==1 else 1).values
acc2_iso['anomaly_iso'] = acc2_iso_scaled['anomaly_iso']
acc2_iso_scaled['anomaly_iso'].value_counts()

In [ ]:
bank_main['iso_2'] = 0
bank_main.loc[acc2_iso_scaled.index,'iso_2'] = acc2_iso_scaled['anomaly_iso']
bank_main['iso_2'].value_counts()

In [ ]:
labels = acc2_iso_scaled.anomaly_iso
colors = {0:'blue', 1:'red'}

fig = plt.figure(1, figsize=(8,8))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(acc2_iso_scaled.iloc[:,3], acc2_iso_scaled.iloc[:,4], acc2_iso_scaled.iloc[:,5], c=labels.apply(lambda x:colors[x]))
ax.set_title(f'ISOLATION FOREST (account_no = {accounts[1]}, outliers_fraction = {100*outliers_fraction}%)', fontsize=14)
ax.set_xlabel('withdrawal_amt')
ax.set_ylabel('deposit_amt')
ax.set_zlabel('balance_amt')

In [ ]:

df = acc2_iso.copy()
a= df.loc[df['anomaly_iso']==1, ['date', 'balance_amt']]
plt.figure(figsize=(20,8))
plt.plot(df['date'], df['balance_amt'], color='blue', label='Normal')
plt.scatter(a['date'], a['balance_amt'], color='red', label='Anomaly')
plt.title(f'{accounts[1]} Isolation Forest Anomalies')
plt.xlabel('Date')
plt.ylabel('balance_amt')
plt.legend()
plt.show()

#### &emsp;(III)  Isolation Forest account_no: 409000438620

In [ ]:
acc3_iso = acc3.copy() 
scaler = StandardScaler()
acc3_iso_scaled = pd.DataFrame(scaler.fit_transform(acc3_iso[acc3.columns[2:]]), columns=acc3.columns[2:])
acc3_iso_scaled.set_index(acc3_iso.index, drop=True, inplace=True)
acc3_iso_scaled.head(3)

In [ ]:
outliers_fraction = .01
model = IsolationForest(contamination=outliers_fraction)
model.fit(acc3_iso_scaled)

In [ ]:
acc3_iso_scaled['anomaly_iso'] = pd.Series(model.predict(acc3_iso_scaled)).apply(lambda x: 0 if x==1 else 1).values
acc3_iso['anomaly_iso'] = acc3_iso_scaled['anomaly_iso']
acc3_iso_scaled['anomaly_iso'].value_counts()

In [ ]:
bank_main['iso_3'] = 0
bank_main.loc[acc3_iso_scaled.index,'iso_3'] = acc3_iso_scaled['anomaly_iso']
bank_main['iso_3'].value_counts()

In [ ]:
labels = acc3_iso_scaled.anomaly_iso
colors = {0:'blue', 1:'red'}

fig = plt.figure(1, figsize=(8,8))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(acc3_iso_scaled.iloc[:,3], acc3_iso_scaled.iloc[:,4], acc3_iso_scaled.iloc[:,5], c=labels.apply(lambda x:colors[x]))
ax.set_title(f'ISOLATION FOREST (account_no = {accounts[2]}, outliers_fraction = {100*outliers_fraction}%)', fontsize=14)
ax.set_xlabel('withdrawal_amt')
ax.set_ylabel('deposit_amt')
ax.set_zlabel('balance_amt')

In [ ]:
df = acc3_iso.copy()
a= df.loc[df['anomaly_iso']==1, ['date', 'balance_amt']]
plt.figure(figsize=(20,8))
plt.plot(df['date'], df['balance_amt'], color='blue', label='Normal')
plt.scatter(a['date'], a['balance_amt'], color='red', label='Anomaly')
plt.title(f'{accounts[2]} Isolation Forest Anomalies')
plt.xlabel('Date')
plt.ylabel('balance_amt')
plt.legend()
plt.show()

#### &emsp;(IV)  Isolation Forest account_no: 1196711

In [ ]:
acc4_iso = acc4.copy() 
scaler = StandardScaler()
acc4_iso_scaled = pd.DataFrame(scaler.fit_transform(acc4_iso[acc4.columns[2:]]), columns=acc4.columns[2:])
acc4_iso_scaled.set_index(acc4_iso.index, drop=True, inplace=True)
acc4_iso_scaled.head(3)

In [ ]:
outliers_fraction = .01
model = IsolationForest(contamination=outliers_fraction)
model.fit(acc4_iso_scaled)

In [ ]:
acc4_iso_scaled['anomaly_iso'] = pd.Series(model.predict(acc4_iso_scaled)).apply(lambda x: 0 if x==1 else 1).values
acc4_iso['anomaly_iso'] = acc4_iso_scaled['anomaly_iso']
acc4_iso_scaled['anomaly_iso'].value_counts()

In [ ]:
bank_main['iso_4'] = 0
bank_main.loc[acc4_iso_scaled.index,'iso_4'] = acc4_iso_scaled['anomaly_iso']
bank_main['iso_4'].value_counts()

In [ ]:
labels = acc4_iso_scaled.anomaly_iso
colors = {0:'blue', 1:'red'}

fig = plt.figure(1, figsize=(8,8))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(acc4_iso_scaled.iloc[:,3], acc4_iso_scaled.iloc[:,4], acc4_iso_scaled.iloc[:,5], c=labels.apply(lambda x:colors[x]))
ax.set_title(f'ISOLATION FOREST (account_no = {accounts[3]}, outliers_fraction = {100*outliers_fraction}%)', fontsize=14)
ax.set_xlabel('withdrawal_amt')
ax.set_ylabel('deposit_amt')
ax.set_zlabel('balance_amt')

In [ ]:
df = acc4_iso.copy()
a= df.loc[df['anomaly_iso']==1, ['date', 'balance_amt']]
plt.figure(figsize=(20,8))
plt.plot(df['date'], df['balance_amt'], color='blue', label='Normal')
plt.scatter(a['date'], a['balance_amt'], color='red', label='Anomaly')
plt.title(f'{accounts[3]} Isolation Forest Anomalies')
plt.xlabel('Date')
plt.ylabel('balance_amt')
plt.legend()
plt.show()

#### &emsp;(V)  Isolation Forest account_no: 409000493210

In [ ]:
acc5_iso = acc5.copy() 
scaler = StandardScaler()
acc5_iso_scaled = pd.DataFrame(scaler.fit_transform(acc5_iso[acc5.columns[2:]]), columns=acc5.columns[2:])
acc5_iso_scaled.set_index(acc5_iso.index, drop=True, inplace=True)
acc5_iso_scaled.head(3)

In [ ]:
outliers_fraction = .01
model = IsolationForest(contamination=outliers_fraction)
model.fit(acc5_iso_scaled)

In [ ]:
acc5_iso_scaled['anomaly_iso'] = pd.Series(model.predict(acc5_iso_scaled)).apply(lambda x: 0 if x==1 else 1).values
acc5_iso['anomaly_iso'] = acc5_iso_scaled['anomaly_iso']
acc5_iso_scaled['anomaly_iso'].value_counts()

In [ ]:
bank_main['iso_5'] = 0
bank_main.loc[acc5_iso_scaled.index,'iso_5'] = acc5_iso_scaled['anomaly_iso']
bank_main['iso_5'].value_counts()

In [ ]:
labels = acc5_iso_scaled.anomaly_iso
colors = {0:'blue', 1:'red'}

fig = plt.figure(1, figsize=(8,8))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(acc5_iso_scaled.iloc[:,3], acc5_iso_scaled.iloc[:,4], acc5_iso_scaled.iloc[:,5], c=labels.apply(lambda x:colors[x]))
ax.set_title(f'ISOLATION FOREST (account_no = {accounts[4]}, outliers_fraction = {100*outliers_fraction}%)', fontsize=14)
ax.set_xlabel('withdrawal_amt')
ax.set_ylabel('deposit_amt')
ax.set_zlabel('balance_amt')

In [ ]:
df = acc5_iso.copy()
a= df.loc[df['anomaly_iso']==1, ['date', 'balance_amt']]
plt.figure(figsize=(20,8))
plt.plot(df['date'], df['balance_amt'], color='blue', label='Normal')
plt.scatter(a['date'], a['balance_amt'], color='red', label='Anomaly')
plt.title(f'{accounts[4]} Isolation Forest Anomalies')
plt.xlabel('Date')
plt.ylabel('balance_amt')
plt.legend()
plt.show()

#### &emsp;(VI)  Isolation Forest account_no: 409000438611

In [ ]:
acc6_iso = acc6.copy() 
scaler = StandardScaler()
acc6_iso_scaled = pd.DataFrame(scaler.fit_transform(acc6_iso[acc6.columns[2:]]), columns=acc6.columns[2:])
acc6_iso_scaled.set_index(acc6_iso.index, drop=True, inplace=True)
acc6_iso_scaled.head(3)

In [ ]:
outliers_fraction = .01
model = IsolationForest(contamination=outliers_fraction)
model.fit(acc6_iso_scaled)

In [ ]:
acc6_iso_scaled['anomaly_iso'] = pd.Series(model.predict(acc6_iso_scaled)).apply(lambda x: 0 if x==1 else 1).values
acc6_iso['anomaly_iso'] = acc6_iso_scaled['anomaly_iso']
acc6_iso_scaled['anomaly_iso'].value_counts()

In [ ]:
bank_main['iso_6'] = 0
bank_main.loc[acc6_iso_scaled.index,'iso_6'] = acc6_iso_scaled['anomaly_iso']
bank_main['iso_6'].value_counts()

In [ ]:
labels = acc6_iso_scaled.anomaly_iso
colors = {0:'blue', 1:'red'}

fig = plt.figure(1, figsize=(8,8))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(acc6_iso_scaled.iloc[:,3], acc6_iso_scaled.iloc[:,4], acc6_iso_scaled.iloc[:,5], c=labels.apply(lambda x:colors[x]))
ax.set_title(f'ISOLATION FOREST (account_no = {accounts[5]}, outliers_fraction = {100*outliers_fraction}%)', fontsize=14)
ax.set_xlabel('withdrawal_amt')
ax.set_ylabel('deposit_amt')
ax.set_zlabel('balance_amt')

In [ ]:
df = acc6_iso.copy()
a= df.loc[df['anomaly_iso']==1, ['date', 'balance_amt']]
plt.figure(figsize=(20,8))
plt.plot(df['date'], df['balance_amt'], color='blue', label='Normal')
plt.scatter(a['date'], a['balance_amt'], color='red', label='Anomaly')
plt.title(f'{accounts[4]} Isolation Forest Anomalies')
plt.xlabel('Date')
plt.ylabel('balance_amt')
plt.legend()
plt.show()

#### &emsp;(VII)  Isolation Forest account_no: 409000611074

In [ ]:
acc7_iso = acc7.copy() 
scaler = StandardScaler()
acc7_iso_scaled = pd.DataFrame(scaler.fit_transform(acc7_iso[acc7.columns[2:]]), columns=acc7.columns[2:])
acc7_iso_scaled.set_index(acc7_iso.index, drop=True, inplace=True)
acc7_iso_scaled.head(3)

In [ ]:
outliers_fraction = .01
model = IsolationForest(contamination=outliers_fraction)
model.fit(acc7_iso_scaled)

In [ ]:
acc7_iso_scaled['anomaly_iso'] = pd.Series(model.predict(acc7_iso_scaled)).apply(lambda x: 0 if x==1 else 1).values
acc7_iso['anomaly_iso'] = acc7_iso_scaled['anomaly_iso']
acc7_iso_scaled['anomaly_iso'].value_counts()

In [ ]:
bank_main['iso_7'] = 0
bank_main.loc[acc7_iso_scaled.index,'iso_7'] = acc7_iso_scaled['anomaly_iso']
bank_main['iso_7'].value_counts()

In [ ]:
labels = acc7_iso_scaled.anomaly_iso
colors = {0:'blue', 1:'red'}

fig = plt.figure(1, figsize=(8,8))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(acc7_iso_scaled.iloc[:,3], acc7_iso_scaled.iloc[:,4], acc7_iso_scaled.iloc[:,5], c=labels.apply(lambda x:colors[x]))
ax.set_title(f'ISOLATION FOREST (account_no = {accounts[6]}, outliers_fraction = {100*outliers_fraction}%)', fontsize=14)
ax.set_xlabel('withdrawal_amt')
ax.set_ylabel('deposit_amt')
ax.set_zlabel('balance_amt')

In [ ]:
df = acc7_iso.copy()
a= df.loc[df['anomaly_iso']==1, ['date', 'balance_amt']]
plt.figure(figsize=(20,8))
plt.plot(df['date'], df['balance_amt'], color='blue', label='Normal')
plt.scatter(a['date'], a['balance_amt'], color='red', label='Anomaly')
plt.title(f'{accounts[6]} Isolation Forest Anomalies')
plt.xlabel('Date')
plt.ylabel('balance_amt')
plt.legend()
plt.show()

#### &emsp;(VIII)  Isolation Forest account_no: 409000493201

In [ ]:
acc8_iso = acc8.copy() 
scaler = StandardScaler()
acc8_iso_scaled = pd.DataFrame(scaler.fit_transform(acc8_iso[acc8.columns[2:]]), columns=acc8.columns[2:])
acc8_iso_scaled.set_index(acc8_iso.index, drop=True, inplace=True)
acc8_iso_scaled.head(3)

In [ ]:
outliers_fraction = .01
model = IsolationForest(contamination=outliers_fraction)
model.fit(acc8_iso_scaled)

In [ ]:
acc8_iso_scaled['anomaly_iso'] = pd.Series(model.predict(acc8_iso_scaled)).apply(lambda x: 0 if x==1 else 1).values
acc8_iso['anomaly_iso'] = acc8_iso_scaled['anomaly_iso']
acc8_iso_scaled['anomaly_iso'].value_counts()

In [ ]:
bank_main['iso_8'] = 0
bank_main.loc[acc8_iso_scaled.index,'iso_8'] = acc8_iso_scaled['anomaly_iso']
bank_main['iso_8'].value_counts()

In [ ]:
labels = acc8_iso_scaled.anomaly_iso
colors = {0:'blue', 1:'red'}

fig = plt.figure(1, figsize=(8,8))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(acc8_iso_scaled.iloc[:,3], acc8_iso_scaled.iloc[:,4], acc8_iso_scaled.iloc[:,5], c=labels.apply(lambda x:colors[x]))
ax.set_title(f'ISOLATION FOREST (account_no = {accounts[7]}, outliers_fraction = {100*outliers_fraction}%)', fontsize=14)
ax.set_xlabel('withdrawal_amt')
ax.set_ylabel('deposit_amt')
ax.set_zlabel('balance_amt')

In [ ]:
df = acc8_iso.copy()
a= df.loc[df['anomaly_iso']==1, ['date', 'balance_amt']]
plt.figure(figsize=(20,8))
plt.plot(df['date'], df['balance_amt'], color='blue', label='Normal')
plt.scatter(a['date'], a['balance_amt'], color='red', label='Anomaly')
plt.title(f'{accounts[7]} Isolation Forest Anomalies')
plt.xlabel('Date')
plt.ylabel('balance_amt')
plt.legend()
plt.show()

#### &emsp;(IX)  Isolation Forest account_no: 409000425051

In [ ]:
acc9_iso = acc9.copy() 
scaler = StandardScaler()
acc9_iso_scaled = pd.DataFrame(scaler.fit_transform(acc9_iso[acc9.columns[2:]]), columns=acc9.columns[2:])
acc9_iso_scaled.set_index(acc9_iso.index, drop=True, inplace=True)
acc9_iso_scaled.head(3)

In [ ]:
outliers_fraction = .01
model = IsolationForest(contamination=outliers_fraction)
model.fit(acc9_iso_scaled)

In [ ]:
acc9_iso_scaled['anomaly_iso'] = pd.Series(model.predict(acc9_iso_scaled)).apply(lambda x: 0 if x==1 else 1).values
acc9_iso['anomaly_iso'] = acc9_iso_scaled['anomaly_iso']
acc9_iso_scaled['anomaly_iso'].value_counts()

In [ ]:
bank_main['iso_9'] = 0
bank_main.loc[acc9_iso_scaled.index,'iso_9'] = acc9_iso_scaled['anomaly_iso']
bank_main['iso_9'].value_counts()

In [ ]:
labels = acc9_iso_scaled.anomaly_iso
colors = {0:'blue', 1:'red'}

fig = plt.figure(1, figsize=(8,8))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(acc9_iso_scaled.iloc[:,3], acc9_iso_scaled.iloc[:,4], acc9_iso_scaled.iloc[:,5], c=labels.apply(lambda x:colors[x]))
ax.set_title(f'ISOLATION FOREST (account_no = {accounts[8]}, outliers_fraction = {100*outliers_fraction}%)', fontsize=14)
ax.set_xlabel('withdrawal_amt')
ax.set_ylabel('deposit_amt')
ax.set_zlabel('balance_amt')

In [ ]:
df = acc9_iso.copy()
a= df.loc[df['anomaly_iso']==1, ['date', 'balance_amt']]
plt.figure(figsize=(20,8))
plt.plot(df['date'], df['balance_amt'], color='blue', label='Normal')
plt.scatter(a['date'], a['balance_amt'], color='red', label='Anomaly')
plt.title(f'{accounts[8]} Isolation Forest Anomalies')
plt.xlabel('Date')
plt.ylabel('balance_amt')
plt.legend()
plt.show()

#### &emsp;(X)  Isolation Forest account_no: 409000405747

In [ ]:
acc10_iso = acc10.copy() 
scaler = StandardScaler()
acc10_iso_scaled = pd.DataFrame(scaler.fit_transform(acc10_iso[acc10.columns[2:]]), columns=acc10.columns[2:])
acc10_iso_scaled.set_index(acc10_iso.index, drop=True, inplace=True)
acc10_iso_scaled.head(3)

In [ ]:
outliers_fraction = .1
model = IsolationForest(contamination=outliers_fraction)
model.fit(acc10_iso_scaled)

In [ ]:
acc10_iso_scaled['anomaly_iso'] = pd.Series(model.predict(acc10_iso_scaled)).apply(lambda x: 0 if x==1 else 1).values
acc10_iso['anomaly_iso'] = acc10_iso_scaled['anomaly_iso']
acc10_iso_scaled['anomaly_iso'].value_counts()

In [ ]:
bank_main['iso_10'] = 0
bank_main.loc[acc10_iso_scaled.index,'iso_10'] = acc10_iso_scaled['anomaly_iso']
bank_main['iso_10'].value_counts()

In [ ]:
labels = acc10_iso_scaled.anomaly_iso
colors = {0:'blue', 1:'red'}

fig = plt.figure(1, figsize=(8,8))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(acc10_iso_scaled.iloc[:,3], acc10_iso_scaled.iloc[:,4], acc10_iso_scaled.iloc[:,5], c=labels.apply(lambda x:colors[x]))
ax.set_title(f'ISOLATION FOREST (account_no = {accounts[9]}, outliers_fraction = {100*outliers_fraction}%)', fontsize=14)
ax.set_xlabel('withdrawal_amt')
ax.set_ylabel('deposit_amt')
ax.set_zlabel('balance_amt')

In [ ]:
df = acc10_iso.copy()
a= df.loc[df['anomaly_iso']==1, ['date', 'balance_amt']]
plt.figure(figsize=(20,8))
plt.plot(df['date'], df['balance_amt'], color='blue', label='Normal')
plt.scatter(a['date'], a['balance_amt'], color='red', label='Anomaly')
plt.title(f'{accounts[9]} Isolation Forest Anomalies')
plt.xlabel('Date')
plt.ylabel('balance_amt')
plt.legend()
plt.show()

In [ ]:
bank_main['accounts_isolation_forest'] = bank_main.iso_1 + bank_main.iso_2 + bank_main.iso_3 + bank_main.iso_4 + bank_main.iso_5 + bank_main.iso_6 + bank_main.iso_7 + bank_main.iso_8 + bank_main.iso_9 + bank_main.iso_10

In [ ]:
#bank_main.drop(columns=['iso_1', 'iso_2', 'iso_3', 'iso_4', 'iso_5', 'iso_6', 'iso_7', 'iso_8', 'iso_9','iso_10'], axis=1, inplace=True)

In [ ]:
bank_main['accounts_isolation_forest'].value_counts()

### ii. OneClassSVM Anomaly Detection

#### &emsp;(I)  OneClassSVM account_no: 1196428

In [ ]:
acc1_svm = acc1.copy() 
scaler = StandardScaler()
acc1_svm_scaled = pd.DataFrame(scaler.fit_transform(acc1_svm[acc1.columns[2:]]), columns=acc1.columns[2:])
acc1_svm_scaled.set_index(acc1_svm.index, drop=True, inplace=True)
acc1_svm_scaled.head(3)

In [ ]:
outliers_fraction = .005
model = OneClassSVM(nu=outliers_fraction, kernel='rbf', gamma=.01)
model.fit(acc1_svm_scaled)

In [ ]:
acc1_svm_scaled['anomaly_svm'] = pd.Series(model.predict(acc1_svm_scaled)).apply(lambda x: 0 if x==1 else 1).values
acc1_svm['anomaly_svm'] = acc1_svm_scaled['anomaly_svm']
acc1_svm_scaled['anomaly_svm'].value_counts()

In [ ]:
bank_main['svm_1'] = 0
bank_main.loc[acc1_svm_scaled.index,'svm_1'] = acc1_svm_scaled['anomaly_svm']
bank_main['svm_1'].value_counts()

In [ ]:
labels = acc1_svm_scaled.anomaly_svm
colors = {0:'blue', 1:'red'}

fig = plt.figure(1, figsize=(8,8))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(acc1_svm_scaled.iloc[:,3], acc1_svm_scaled.iloc[:,4], acc1_svm_scaled.iloc[:,5], c=labels.apply(lambda x:colors[x]))
ax.set_title(f'ONE CLASS SVM (account_no = {accounts[0]}, outliers_fraction = {100*outliers_fraction}%)', fontsize=14)
ax.set_xlabel('withdrawal_amt')
ax.set_ylabel('deposit_amt')
ax.set_zlabel('balance_amt')

In [ ]:
df = acc1_svm.copy()
a= df.loc[df['anomaly_svm']==1, ['date', 'balance_amt']]
plt.figure(figsize=(20,8))
plt.plot(df['date'], df['balance_amt'], color='blue', label='Normal')
plt.scatter(a['date'], a['balance_amt'], color='red', label='Anomaly')
plt.title(f'{accounts[0]} OneClassSVM Anomalies')
plt.xlabel('Date')
plt.ylabel('balance_amt')
plt.legend()
plt.show()

#### &emsp;(II)  OneClassSVM account_no: 409000362497

In [ ]:
acc2_svm = acc2.copy() 
scaler = StandardScaler()
acc2_svm_scaled = pd.DataFrame(scaler.fit_transform(acc2_svm[acc2.columns[2:]]), columns=acc2.columns[2:])
acc2_svm_scaled.set_index(acc2_svm.index, drop=True, inplace=True)
acc2_svm_scaled.head(3)

In [ ]:
outliers_fraction = .005
model = OneClassSVM(nu=outliers_fraction, kernel='rbf', gamma=.01)
model.fit(acc2_svm_scaled)

In [ ]:
acc2_svm_scaled['anomaly_svm'] = pd.Series(model.predict(acc2_svm_scaled)).apply(lambda x: 0 if x==1 else 1).values
acc2_svm['anomaly_svm'] = acc2_svm_scaled['anomaly_svm']
acc2_svm_scaled['anomaly_svm'].value_counts()

In [ ]:
bank_main['svm_2'] = 0
bank_main.loc[acc2_svm_scaled.index,'svm_2'] = acc2_svm_scaled['anomaly_svm']
bank_main['svm_2'].value_counts()

In [ ]:
labels = acc2_svm_scaled.anomaly_svm
colors = {0:'blue', 1:'red'}

fig = plt.figure(1, figsize=(8,8))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(acc2_svm_scaled.iloc[:,3], acc2_svm_scaled.iloc[:,4], acc2_svm_scaled.iloc[:,5], c=labels.apply(lambda x:colors[x]))
ax.set_title(f'ONE CLASS SVM (account_no = {accounts[1]}, outliers_fraction = {100*outliers_fraction}%)', fontsize=14)
ax.set_xlabel('withdrawal_amt')
ax.set_ylabel('deposit_amt')
ax.set_zlabel('balance_amt')

In [ ]:
df = acc2_svm.copy()
a= df.loc[df['anomaly_svm']==1, ['date', 'balance_amt']]
plt.figure(figsize=(20,8))
plt.plot(df['date'], df['balance_amt'], color='blue', label='Normal')
plt.scatter(a['date'], a['balance_amt'], color='red', label='Anomaly')
plt.title(f'{accounts[1]} OneClassSVM Anomalies')
plt.xlabel('Date')
plt.ylabel('balance_amt')
plt.legend()
plt.show()

#### &emsp;(III)  OneClassSVM account_no: 409000438620

In [ ]:
acc3_svm = acc3.copy() 
scaler = StandardScaler()
acc3_svm_scaled = pd.DataFrame(scaler.fit_transform(acc3_svm[acc3.columns[2:]]), columns=acc3.columns[2:])
acc3_svm_scaled.set_index(acc3_svm.index, drop=True, inplace=True)
acc3_svm_scaled.head(3)

In [ ]:
outliers_fraction = .005
model = OneClassSVM(nu=outliers_fraction, kernel='rbf', gamma=.01)
model.fit(acc3_svm_scaled)

In [ ]:
acc3_svm_scaled['anomaly_svm'] = pd.Series(model.predict(acc3_svm_scaled)).apply(lambda x: 0 if x==1 else 1).values
acc3_svm['anomaly_svm'] = acc3_svm_scaled['anomaly_svm']
acc3_svm_scaled['anomaly_svm'].value_counts()

In [ ]:
bank_main['svm_3'] = 0
bank_main.loc[acc3_svm_scaled.index,'svm_3'] = acc3_svm_scaled['anomaly_svm']
bank_main['svm_3'].value_counts()

In [ ]:
labels = acc3_svm_scaled.anomaly_svm
colors = {0:'blue', 1:'red'}

fig = plt.figure(1, figsize=(8,8))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(acc3_svm_scaled.iloc[:,3], acc3_svm_scaled.iloc[:,4], acc3_svm_scaled.iloc[:,5], c=labels.apply(lambda x:colors[x]))
ax.set_title(f'ONE CLASS SVM (account_no = {accounts[2]}, outliers_fraction = {100*outliers_fraction}%)', fontsize=14)
ax.set_xlabel('withdrawal_amt')
ax.set_ylabel('deposit_amt')
ax.set_zlabel('balance_amt')

In [ ]:
df = acc3_svm.copy()
a= df.loc[df['anomaly_svm']==1, ['date', 'balance_amt']]
plt.figure(figsize=(20,8))
plt.plot(df['date'], df['balance_amt'], color='blue', label='Normal')
plt.scatter(a['date'], a['balance_amt'], color='red', label='Anomaly')
plt.title(f'{accounts[2]} OneClassSVM Anomalies')
plt.xlabel('Date')
plt.ylabel('balance_amt')
plt.legend()
plt.show()

#### &emsp;(IV)  OneClassSVM account_no: 1196711

In [ ]:
acc4_svm = acc4.copy() 
scaler = StandardScaler()
acc4_svm_scaled = pd.DataFrame(scaler.fit_transform(acc4_svm[acc4.columns[2:]]), columns=acc4.columns[2:])
acc4_svm_scaled.set_index(acc4_svm.index, drop=True, inplace=True)
acc4_svm_scaled.head(3)

In [ ]:
outliers_fraction = .005
model = OneClassSVM(nu=outliers_fraction, kernel='rbf', gamma=.01)
model.fit(acc4_svm_scaled)

In [ ]:
acc4_svm_scaled['anomaly_svm'] = pd.Series(model.predict(acc4_svm_scaled)).apply(lambda x: 0 if x==1 else 1).values
acc4_svm['anomaly_svm'] = acc4_svm_scaled['anomaly_svm']
acc4_svm_scaled['anomaly_svm'].value_counts()

In [ ]:
bank_main['svm_4'] = 0
bank_main.loc[acc4_svm_scaled.index,'svm_4'] = acc4_svm_scaled['anomaly_svm']
bank_main['svm_4'].value_counts()

In [ ]:
labels = acc4_svm_scaled.anomaly_svm
colors = {0:'blue', 1:'red'}

fig = plt.figure(1, figsize=(8,8))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(acc4_svm_scaled.iloc[:,3], acc4_svm_scaled.iloc[:,4], acc4_svm_scaled.iloc[:,5], c=labels.apply(lambda x:colors[x]))
ax.set_title(f'ONE CLASS SVM (account_no = {accounts[3]}, outliers_fraction = {100*outliers_fraction}%)', fontsize=14)
ax.set_xlabel('withdrawal_amt')
ax.set_ylabel('deposit_amt')
ax.set_zlabel('balance_amt')

In [ ]:
df = acc4_svm.copy()
a= df.loc[df['anomaly_svm']==1, ['date', 'balance_amt']]
plt.figure(figsize=(20,8))
plt.plot(df['date'], df['balance_amt'], color='blue', label='Normal')
plt.scatter(a['date'], a['balance_amt'], color='red', label='Anomaly')
plt.title(f'{accounts[3]} OneClassSVM Anomalies')
plt.xlabel('Date')
plt.ylabel('balance_amt')
plt.legend()
plt.show()

#### &emsp;(V)  OneClassSVM account_no: 409000493210

In [ ]:
acc5_svm = acc5.copy() 
scaler = StandardScaler()
acc5_svm_scaled = pd.DataFrame(scaler.fit_transform(acc5_svm[acc5.columns[2:]]), columns=acc5.columns[2:])
acc5_svm_scaled.set_index(acc5_svm.index, drop=True, inplace=True)
acc5_svm_scaled.head(3)

In [ ]:
outliers_fraction = .005
model = OneClassSVM(nu=outliers_fraction, kernel='rbf', gamma=.01)
model.fit(acc5_svm_scaled)

In [ ]:
acc5_svm_scaled['anomaly_svm'] = pd.Series(model.predict(acc5_svm_scaled)).apply(lambda x: 0 if x==1 else 1).values
acc5_svm['anomaly_svm'] = acc5_svm_scaled['anomaly_svm']
acc5_svm_scaled['anomaly_svm'].value_counts()

In [ ]:
bank_main['svm_5'] = 0
bank_main.loc[acc5_svm_scaled.index,'svm_5'] = acc5_svm_scaled['anomaly_svm']
bank_main['svm_5'].value_counts()

In [ ]:
labels = acc5_svm_scaled.anomaly_svm
colors = {0:'blue', 1:'red'}

fig = plt.figure(1, figsize=(8,8))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(acc5_svm_scaled.iloc[:,3], acc5_svm_scaled.iloc[:,4], acc5_svm_scaled.iloc[:,5], c=labels.apply(lambda x:colors[x]))
ax.set_title(f'ONE CLASS SVM (account_no = {accounts[4]}, outliers_fraction = {100*outliers_fraction}%)', fontsize=14)
ax.set_xlabel('withdrawal_amt')
ax.set_ylabel('deposit_amt')
ax.set_zlabel('balance_amt')

In [ ]:
df = acc5_svm.copy()
a= df.loc[df['anomaly_svm']==1, ['date', 'balance_amt']]
plt.figure(figsize=(20,8))
plt.plot(df['date'], df['balance_amt'], color='blue', label='Normal')
plt.scatter(a['date'], a['balance_amt'], color='red', label='Anomaly')
plt.title(f'{accounts[4]} OneClassSVM Anomalies')
plt.xlabel('Date')
plt.ylabel('balance_amt')
plt.legend()
plt.show()

#### &emsp;(VI)  OneClassSVM account_no: 409000438611

In [ ]:
acc6_svm = acc6.copy() 
scaler = StandardScaler()
acc6_svm_scaled = pd.DataFrame(scaler.fit_transform(acc6_svm[acc6.columns[2:]]), columns=acc6.columns[2:])
acc6_svm_scaled.set_index(acc6_svm.index, drop=True, inplace=True)
acc6_svm_scaled.head(3)

In [ ]:
outliers_fraction = .005
model = OneClassSVM(nu=outliers_fraction, kernel='rbf', gamma=.01)
model.fit(acc6_svm_scaled)

In [ ]:
acc6_svm_scaled['anomaly_svm'] = pd.Series(model.predict(acc6_svm_scaled)).apply(lambda x: 0 if x==1 else 1).values
acc6_svm['anomaly_svm'] = acc6_svm_scaled['anomaly_svm']
acc6_svm_scaled['anomaly_svm'].value_counts()

In [ ]:
bank_main['svm_6'] = 0
bank_main.loc[acc6_svm_scaled.index,'svm_6'] = acc6_svm_scaled['anomaly_svm']
bank_main['svm_6'].value_counts()

In [ ]:
labels = acc6_svm_scaled.anomaly_svm
colors = {0:'blue', 1:'red'}

fig = plt.figure(1, figsize=(8,8))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(acc6_svm_scaled.iloc[:,3], acc6_svm_scaled.iloc[:,4], acc6_svm_scaled.iloc[:,5], c=labels.apply(lambda x:colors[x]))
ax.set_title(f'ONE CLASS SVM (account_no = {accounts[5]}, outliers_fraction = {100*outliers_fraction}%)', fontsize=14)
ax.set_xlabel('withdrawal_amt')
ax.set_ylabel('deposit_amt')
ax.set_zlabel('balance_amt')

In [ ]:
df = acc6_svm.copy()
a= df.loc[df['anomaly_svm']==1, ['date', 'balance_amt']]
plt.figure(figsize=(20,8))
plt.plot(df['date'], df['balance_amt'], color='blue', label='Normal')
plt.scatter(a['date'], a['balance_amt'], color='red', label='Anomaly')
plt.title(f'{accounts[5]} OneClassSVM Anomalies')
plt.xlabel('Date')
plt.ylabel('balance_amt')
plt.legend()
plt.show()

#### &emsp;(VII)  OneClassSVM account_no: 409000611074

In [ ]:
acc7_svm = acc7.copy() 
scaler = StandardScaler()
acc7_svm_scaled = pd.DataFrame(scaler.fit_transform(acc7_svm[acc7.columns[2:]]), columns=acc7.columns[2:])
acc7_svm_scaled.set_index(acc7_svm.index, drop=True, inplace=True)
acc7_svm_scaled.head(3)

In [ ]:
outliers_fraction = .01
model = OneClassSVM(nu=outliers_fraction, kernel='rbf', gamma=.01)
model.fit(acc7_svm_scaled)

In [ ]:
acc7_svm_scaled['anomaly_svm'] = pd.Series(model.predict(acc7_svm_scaled)).apply(lambda x: 0 if x==1 else 1).values
acc7_svm['anomaly_svm'] = acc7_svm_scaled['anomaly_svm']
acc7_svm_scaled['anomaly_svm'].value_counts()

In [ ]:
bank_main['svm_7'] = 0
bank_main.loc[acc7_svm_scaled.index,'svm_7'] = acc7_svm_scaled['anomaly_svm']
bank_main['svm_7'].value_counts()

In [ ]:
labels = acc7_svm_scaled.anomaly_svm
colors = {0:'blue', 1:'red'}

fig = plt.figure(1, figsize=(8,8))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(acc7_svm_scaled.iloc[:,3], acc7_svm_scaled.iloc[:,4], acc7_svm_scaled.iloc[:,5], c=labels.apply(lambda x:colors[x]))
ax.set_title(f'ONE CLASS SVM (account_no = {accounts[6]}, outliers_fraction = {100*outliers_fraction}%)', fontsize=14)
ax.set_xlabel('withdrawal_amt')
ax.set_ylabel('deposit_amt')
ax.set_zlabel('balance_amt')

In [ ]:
df = acc7_svm.copy()
a= df.loc[df['anomaly_svm']==1, ['date', 'balance_amt']]
plt.figure(figsize=(20,8))
plt.plot(df['date'], df['balance_amt'], color='blue', label='Normal')
plt.scatter(a['date'], a['balance_amt'], color='red', label='Anomaly')
plt.title(f'{accounts[6]} OneClassSVM Anomalies')
plt.xlabel('Date')
plt.ylabel('balance_amt')
plt.legend()
plt.show()

#### &emsp;(VIII)  OneClassSVM account_no: 409000493201

In [ ]:
acc8_svm = acc8.copy() 
scaler = StandardScaler()
acc8_svm_scaled = pd.DataFrame(scaler.fit_transform(acc8_svm[acc8.columns[2:]]), columns=acc8.columns[2:])
acc8_svm_scaled.set_index(acc8_svm.index, drop=True, inplace=True)
acc8_svm_scaled.head(3)

In [ ]:
outliers_fraction = .01
model = OneClassSVM(nu=outliers_fraction, kernel='rbf', gamma=.01)
model.fit(acc8_svm_scaled)

In [ ]:
acc8_svm_scaled['anomaly_svm'] = pd.Series(model.predict(acc8_svm_scaled)).apply(lambda x: 0 if x==1 else 1).values
acc8_svm['anomaly_svm'] = acc8_svm_scaled['anomaly_svm']
acc8_svm_scaled['anomaly_svm'].value_counts()

In [ ]:
bank_main['svm_8'] = 0
bank_main.loc[acc8_svm_scaled.index,'svm_8'] = acc8_svm_scaled['anomaly_svm']
bank_main['svm_8'].value_counts()

In [ ]:
labels = acc8_svm_scaled.anomaly_svm
colors = {0:'blue', 1:'red'}

fig = plt.figure(1, figsize=(8,8))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(acc8_svm_scaled.iloc[:,3], acc8_svm_scaled.iloc[:,4], acc8_svm_scaled.iloc[:,5], c=labels.apply(lambda x:colors[x]))
ax.set_title(f'ONE CLASS SVM (account_no = {accounts[7]}, outliers_fraction = {100*outliers_fraction}%)', fontsize=14)
ax.set_xlabel('withdrawal_amt')
ax.set_ylabel('deposit_amt')
ax.set_zlabel('balance_amt')

In [ ]:
df = acc8_svm.copy()
a= df.loc[df['anomaly_svm']==1, ['date', 'balance_amt']]
plt.figure(figsize=(20,8))
plt.plot(df['date'], df['balance_amt'], color='blue', label='Normal')
plt.scatter(a['date'], a['balance_amt'], color='red', label='Anomaly')
plt.title(f'{accounts[7]} OneClassSVM Anomalies')
plt.xlabel('Date')
plt.ylabel('balance_amt')
plt.legend()
plt.show()

#### &emsp;(IX)  OneClassSVM account_no: 409000425051

In [ ]:
acc9_svm = acc9.copy() 
scaler = StandardScaler()
acc9_svm_scaled = pd.DataFrame(scaler.fit_transform(acc9_svm[acc9.columns[2:]]), columns=acc9.columns[2:])
acc9_svm_scaled.set_index(acc9_svm.index, drop=True, inplace=True)
acc9_svm_scaled.head(3)

In [ ]:
outliers_fraction = .01
model = OneClassSVM(nu=outliers_fraction, kernel='rbf', gamma=.01)
model.fit(acc9_svm_scaled)

In [ ]:
acc9_svm_scaled['anomaly_svm'] = pd.Series(model.predict(acc9_svm_scaled)).apply(lambda x: 0 if x==1 else 1).values
acc9_svm['anomaly_svm'] = acc9_svm_scaled['anomaly_svm']
acc9_svm_scaled['anomaly_svm'].value_counts()

In [ ]:
bank_main['svm_9'] = 0
bank_main.loc[acc9_svm_scaled.index,'svm_9'] = acc9_svm_scaled['anomaly_svm']
bank_main['svm_9'].value_counts()

In [ ]:
labels = acc9_svm_scaled.anomaly_svm
colors = {0:'blue', 1:'red'}

fig = plt.figure(1, figsize=(8,8))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(acc9_svm_scaled.iloc[:,3], acc9_svm_scaled.iloc[:,4], acc9_svm_scaled.iloc[:,5], c=labels.apply(lambda x:colors[x]))
ax.set_title(f'ONE CLASS SVM (account_no = {accounts[8]}, outliers_fraction = {100*outliers_fraction}%)', fontsize=14)
ax.set_xlabel('withdrawal_amt')
ax.set_ylabel('deposit_amt')
ax.set_zlabel('balance_amt')

In [ ]:
df = acc9_svm.copy()
a= df.loc[df['anomaly_svm']==1, ['date', 'balance_amt']]
plt.figure(figsize=(20,8))
plt.plot(df['date'], df['balance_amt'], color='blue', label='Normal')
plt.scatter(a['date'], a['balance_amt'], color='red', label='Anomaly')
plt.title(f'{accounts[8]} OneClassSVM Anomalies')
plt.xlabel('Date')
plt.ylabel('balance_amt')
plt.legend()
plt.show()

#### &emsp;(X)  OneClassSVM account_no: 409000405747

In [ ]:
acc10_svm = acc10.copy() 
scaler = StandardScaler()
acc10_svm_scaled = pd.DataFrame(scaler.fit_transform(acc10_svm[acc10.columns[2:]]), columns=acc10.columns[2:])
acc10_svm_scaled.set_index(acc10_svm.index, drop=True, inplace=True)
acc10_svm_scaled.head(3)

In [ ]:
outliers_fraction = .09
model = OneClassSVM(nu=outliers_fraction, kernel='rbf', gamma=.01)
model.fit(acc10_svm_scaled)

In [ ]:
acc10_svm_scaled['anomaly_svm'] = pd.Series(model.predict(acc10_svm_scaled)).apply(lambda x: 0 if x==1 else 1).values
acc10_svm['anomaly_svm'] = acc10_svm_scaled['anomaly_svm']
acc10_svm_scaled['anomaly_svm'].value_counts()

In [ ]:
bank_main['svm_10'] = 0
bank_main.loc[acc10_svm_scaled.index,'svm_10'] = acc10_svm_scaled['anomaly_svm']
bank_main['svm_10'].value_counts()

In [ ]:
labels = acc10_svm_scaled.anomaly_svm
colors = {0:'blue', 1:'red'}

fig = plt.figure(1, figsize=(8,8))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(acc10_svm_scaled.iloc[:,3], acc10_svm_scaled.iloc[:,4], acc10_svm_scaled.iloc[:,5], c=labels.apply(lambda x:colors[x]))
ax.set_title(f'ONE CLASS SVM (account_no = {accounts[9]}, outliers_fraction = {100*outliers_fraction}%)', fontsize=14)
ax.set_xlabel('withdrawal_amt')
ax.set_ylabel('deposit_amt')
ax.set_zlabel('balance_amt')

In [ ]:
df = acc10_svm.copy()
a= df.loc[df['anomaly_svm']==1, ['date', 'balance_amt']]
plt.figure(figsize=(20,8))
plt.plot(df['date'], df['balance_amt'], color='blue', label='Normal')
plt.scatter(a['date'], a['balance_amt'], color='red', label='Anomaly')
plt.title(f'{accounts[9]} OneClassSVM Anomalies')
plt.xlabel('Date')
plt.ylabel('balance_amt')
plt.legend()
plt.show()

In [ ]:
bank_main['accounts_SVM'] = bank_main.svm_1 + bank_main.svm_2 + bank_main.svm_3 + bank_main.svm_4 + bank_main.svm_5 + bank_main.svm_6 + bank_main.svm_7 + bank_main.svm_8 + bank_main.svm_9 + bank_main.svm_10

In [ ]:
bank_main['accounts_SVM'].value_counts()

### iii. Gaussian Distribution Anomaly Detection (Elliptic Envelope)

#### &emsp;(I)  Gaussian Distribution account_no: 409000405747

In [ ]:
acc1_gauss = acc1.copy() 
scaler = StandardScaler()
acc1_gauss_scaled = pd.DataFrame(scaler.fit_transform(acc1_gauss[acc1.columns[2:]]), columns=acc1.columns[2:])
acc1_gauss_scaled.set_index(acc1_gauss.index, drop=True, inplace=True)
acc1_gauss_scaled.head(3)

In [ ]:
outliers_fraction = .005
model = EllipticEnvelope(contamination=outliers_fraction)
model.fit(acc1_gauss_scaled)

In [ ]:
acc1_gauss_scaled['anomaly_gauss'] = pd.Series(model.predict(acc1_gauss_scaled)).apply(lambda x: 0 if x==1 else 1).values
acc1_gauss['anomaly_gauss'] = acc1_gauss_scaled['anomaly_gauss']
acc1_gauss_scaled['anomaly_gauss'].value_counts()

In [ ]:
bank_main['gauss_1'] = 0
bank_main.loc[acc1_gauss_scaled.index,'gauss_1'] = acc1_gauss_scaled['anomaly_gauss']
bank_main['gauss_1'].value_counts()

In [ ]:
labels = acc1_gauss_scaled.anomaly_gauss
colors = {0:'blue', 1:'red'}

fig = plt.figure(1, figsize=(8,8))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(acc1_gauss_scaled.iloc[:,3], acc1_gauss_scaled.iloc[:,4], acc1_gauss_scaled.iloc[:,5], c=labels.apply(lambda x:colors[x]))
ax.set_title(f'GAUSSIAN DISTRIBUTION (account_no = {accounts[0]}, outliers_fraction = {100*outliers_fraction}%)', fontsize=14)
ax.set_xlabel('withdrawal_amt')
ax.set_ylabel('deposit_amt')
ax.set_zlabel('balance_amt')

In [ ]:
df = acc1_gauss.copy()
a= df.loc[df['anomaly_gauss']==1, ['date', 'balance_amt']]
plt.figure(figsize=(20,8))
plt.plot(df['date'], df['balance_amt'], color='blue', label='Normal')
plt.scatter(a['date'], a['balance_amt'], color='red', label='Anomaly')
plt.title(f'{accounts[0]} Gaussian Distribution Anomalies')
plt.xlabel('Date')
plt.ylabel('balance_amt')
plt.legend()
plt.show()

#### &emsp;(II)  Gaussian Distribution account_no: 409000362497

In [ ]:
acc2_gauss = acc2.copy() 
scaler = StandardScaler()
acc2_gauss_scaled = pd.DataFrame(scaler.fit_transform(acc2_gauss[acc2.columns[2:]]), columns=acc2.columns[2:])
acc2_gauss_scaled.set_index(acc2_gauss.index, drop=True, inplace=True)
acc2_gauss_scaled.head(3)

In [ ]:
outliers_fraction = .005
model = EllipticEnvelope(contamination=outliers_fraction)
model.fit(acc2_gauss_scaled)

In [ ]:
acc2_gauss_scaled['anomaly_gauss'] = pd.Series(model.predict(acc2_gauss_scaled)).apply(lambda x: 0 if x==1 else 1).values
acc2_gauss['anomaly_gauss'] = acc2_gauss_scaled['anomaly_gauss']
acc2_gauss_scaled['anomaly_gauss'].value_counts()

In [ ]:
bank_main['gauss_2'] = 0
bank_main.loc[acc2_gauss_scaled.index,'gauss_2'] = acc2_gauss_scaled['anomaly_gauss']
bank_main['gauss_2'].value_counts()

In [ ]:
labels = acc2_gauss_scaled.anomaly_gauss
colors = {0:'blue', 1:'red'}

fig = plt.figure(1, figsize=(8,8))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(acc2_gauss_scaled.iloc[:,3], acc2_gauss_scaled.iloc[:,4], acc2_gauss_scaled.iloc[:,5], c=labels.apply(lambda x:colors[x]))
ax.set_title(f'GAUSSIAN DISTRIBUTION (account_no = {accounts[1]}, outliers_fraction = {100*outliers_fraction}%)', fontsize=14)
ax.set_xlabel('withdrawal_amt')
ax.set_ylabel('deposit_amt')
ax.set_zlabel('balance_amt')

In [ ]:
df = acc2_gauss.copy()
a= df.loc[df['anomaly_gauss']==1, ['date', 'balance_amt']]
plt.figure(figsize=(20,8))
plt.plot(df['date'], df['balance_amt'], color='blue', label='Normal')
plt.scatter(a['date'], a['balance_amt'], color='red', label='Anomaly')
plt.title(f'{accounts[1]} Gaussian Distribution Anomalies')
plt.xlabel('Date')
plt.ylabel('balance_amt')
plt.legend()
plt.show()

#### &emsp;(III)  Gaussian Distribution account_no: 409000438620

In [ ]:
acc3_gauss = acc3.copy() 
scaler = StandardScaler()
acc3_gauss_scaled = pd.DataFrame(scaler.fit_transform(acc3_gauss[acc3.columns[2:]]), columns=acc3.columns[2:])
acc3_gauss_scaled.set_index(acc3_gauss.index, drop=True, inplace=True)
acc3_gauss_scaled.head(3)

In [ ]:
outliers_fraction = .005
model = EllipticEnvelope(contamination=outliers_fraction)
model.fit(acc3_gauss_scaled)

In [ ]:
acc3_gauss_scaled['anomaly_gauss'] = pd.Series(model.predict(acc3_gauss_scaled)).apply(lambda x: 0 if x==1 else 1).values
acc3_gauss['anomaly_gauss'] = acc3_gauss_scaled['anomaly_gauss']
acc3_gauss_scaled['anomaly_gauss'].value_counts()

In [ ]:
bank_main['gauss_3'] = 0
bank_main.loc[acc3_gauss_scaled.index,'gauss_3'] = acc3_gauss_scaled['anomaly_gauss']
bank_main['gauss_3'].value_counts()

In [ ]:
labels = acc3_gauss_scaled.anomaly_gauss
colors = {0:'blue', 1:'red'}

fig = plt.figure(1, figsize=(8,8))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(acc3_gauss_scaled.iloc[:,3], acc3_gauss_scaled.iloc[:,4], acc3_gauss_scaled.iloc[:,5], c=labels.apply(lambda x:colors[x]))
ax.set_title(f'GAUSSIAN DISTRIBUTION (account_no = {accounts[2]}, outliers_fraction = {100*outliers_fraction}%)', fontsize=14)
ax.set_xlabel('withdrawal_amt')
ax.set_ylabel('deposit_amt')
ax.set_zlabel('balance_amt')

In [ ]:
df = acc3_gauss.copy()
a= df.loc[df['anomaly_gauss']==1, ['date', 'balance_amt']]
plt.figure(figsize=(20,8))
plt.plot(df['date'], df['balance_amt'], color='blue', label='Normal')
plt.scatter(a['date'], a['balance_amt'], color='red', label='Anomaly')
plt.title(f'{accounts[2]} Gaussian Distribution Anomalies')
plt.xlabel('Date')
plt.ylabel('balance_amt')
plt.legend()
plt.show()

#### &emsp;(IV)  Gaussian Distribution account_no: 1196711

In [ ]:
acc4_gauss = acc4.copy() 
scaler = StandardScaler()
acc4_gauss_scaled = pd.DataFrame(scaler.fit_transform(acc4_gauss[acc4.columns[2:]]), columns=acc4.columns[2:])
acc4_gauss_scaled.set_index(acc4_gauss.index, drop=True, inplace=True)
acc4_gauss_scaled.head(3)

In [ ]:
outliers_fraction = .005
model = EllipticEnvelope(contamination=outliers_fraction)
model.fit(acc4_gauss_scaled)

In [ ]:
acc4_gauss_scaled['anomaly_gauss'] = pd.Series(model.predict(acc4_gauss_scaled)).apply(lambda x: 0 if x==1 else 1).values
acc4_gauss['anomaly_gauss'] = acc4_gauss_scaled['anomaly_gauss']
acc4_gauss_scaled['anomaly_gauss'].value_counts()

In [ ]:
bank_main['gauss_4'] = 0
bank_main.loc[acc4_gauss_scaled.index,'gauss_4'] = acc4_gauss_scaled['anomaly_gauss']
bank_main['gauss_4'].value_counts()

In [ ]:
labels = acc4_gauss_scaled.anomaly_gauss
colors = {0:'blue', 1:'red'}

fig = plt.figure(1, figsize=(8,8))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(acc4_gauss_scaled.iloc[:,3], acc4_gauss_scaled.iloc[:,4], acc4_gauss_scaled.iloc[:,5], c=labels.apply(lambda x:colors[x]))
ax.set_title(f'GAUSSIAN DISTRIBUTION (account_no = {accounts[3]}, outliers_fraction = {100*outliers_fraction}%)', fontsize=14)
ax.set_xlabel('withdrawal_amt')
ax.set_ylabel('deposit_amt')
ax.set_zlabel('balance_amt')

In [ ]:
df = acc4_gauss.copy()
a= df.loc[df['anomaly_gauss']==1, ['date', 'balance_amt']]
plt.figure(figsize=(20,8))
plt.plot(df['date'], df['balance_amt'], color='blue', label='Normal')
plt.scatter(a['date'], a['balance_amt'], color='red', label='Anomaly')
plt.title(f'{accounts[3]} Gaussian Distribution Anomalies')
plt.xlabel('Date')
plt.ylabel('balance_amt')
plt.legend()
plt.show()

#### &emsp;(V)  Gaussian Distribution account_no: 409000493210

In [ ]:
acc5_gauss = acc5.copy() 
scaler = StandardScaler()
acc5_gauss_scaled = pd.DataFrame(scaler.fit_transform(acc5_gauss[acc5.columns[2:]]), columns=acc5.columns[2:])
acc5_gauss_scaled.set_index(acc5_gauss.index, drop=True, inplace=True)
acc5_gauss_scaled.head(3)

In [ ]:
outliers_fraction = .005
model = EllipticEnvelope(contamination=outliers_fraction)
model.fit(acc5_gauss_scaled)

In [ ]:
acc5_gauss_scaled['anomaly_gauss'] = pd.Series(model.predict(acc5_gauss_scaled)).apply(lambda x: 0 if x==1 else 1).values
acc5_gauss['anomaly_gauss'] = acc5_gauss_scaled['anomaly_gauss']
acc5_gauss_scaled['anomaly_gauss'].value_counts()

In [ ]:
bank_main['gauss_5'] = 0
bank_main.loc[acc5_gauss_scaled.index,'gauss_5'] = acc5_gauss_scaled['anomaly_gauss']
bank_main['gauss_5'].value_counts()

In [ ]:
labels = acc5_gauss_scaled.anomaly_gauss
colors = {0:'blue', 1:'red'}

fig = plt.figure(1, figsize=(8,8))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(acc5_gauss_scaled.iloc[:,3], acc5_gauss_scaled.iloc[:,4], acc5_gauss_scaled.iloc[:,5], c=labels.apply(lambda x:colors[x]))
ax.set_title(f'GAUSSIAN DISTRIBUTION (account_no = {accounts[4]}, outliers_fraction = {100*outliers_fraction}%)', fontsize=14)
ax.set_xlabel('withdrawal_amt')
ax.set_ylabel('deposit_amt')
ax.set_zlabel('balance_amt')

In [ ]:
df = acc5_gauss.copy()
a= df.loc[df['anomaly_gauss']==1, ['date', 'balance_amt']]
plt.figure(figsize=(20,8))
plt.plot(df['date'], df['balance_amt'], color='blue', label='Normal')
plt.scatter(a['date'], a['balance_amt'], color='red', label='Anomaly')
plt.title(f'{accounts[4]} Gaussian Distribution Anomalies')
plt.xlabel('Date')
plt.ylabel('balance_amt')
plt.legend()
plt.show()

#### &emsp;(VI)  Gaussian Distribution account_no: 409000438611

In [ ]:
acc6_gauss = acc6.copy() 
scaler = StandardScaler()
acc6_gauss_scaled = pd.DataFrame(scaler.fit_transform(acc6_gauss[acc6.columns[2:]]), columns=acc6.columns[2:])
acc6_gauss_scaled.set_index(acc6_gauss.index, drop=True, inplace=True)
acc6_gauss_scaled.head(3)

In [ ]:
outliers_fraction = .005
model = EllipticEnvelope(contamination=outliers_fraction)
model.fit(acc6_gauss_scaled)

In [ ]:
acc6_gauss_scaled['anomaly_gauss'] = pd.Series(model.predict(acc6_gauss_scaled)).apply(lambda x: 0 if x==1 else 1).values
acc6_gauss['anomaly_gauss'] = acc6_gauss_scaled['anomaly_gauss']
acc6_gauss_scaled['anomaly_gauss'].value_counts()

In [ ]:
bank_main['gauss_6'] = 0
bank_main.loc[acc6_gauss_scaled.index,'gauss_6'] = acc6_gauss_scaled['anomaly_gauss']
bank_main['gauss_6'].value_counts()

In [ ]:
labels = acc6_gauss_scaled.anomaly_gauss
colors = {0:'blue', 1:'red'}

fig = plt.figure(1, figsize=(8,8))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(acc6_gauss_scaled.iloc[:,3], acc6_gauss_scaled.iloc[:,4], acc6_gauss_scaled.iloc[:,5], c=labels.apply(lambda x:colors[x]))
ax.set_title(f'GAUSSIAN DISTRIBUTION (account_no = {accounts[5]}, outliers_fraction = {100*outliers_fraction}%)', fontsize=14)
ax.set_xlabel('withdrawal_amt')
ax.set_ylabel('deposit_amt')
ax.set_zlabel('balance_amt')

In [ ]:
df = acc6_gauss.copy()
a= df.loc[df['anomaly_gauss']==1, ['date', 'balance_amt']]
plt.figure(figsize=(20,8))
plt.plot(df['date'], df['balance_amt'], color='blue', label='Normal')
plt.scatter(a['date'], a['balance_amt'], color='red', label='Anomaly')
plt.title(f'{accounts[5]} Gaussian Distribution Anomalies')
plt.xlabel('Date')
plt.ylabel('balance_amt')
plt.legend()
plt.show()

#### &emsp;(VII)  Gaussian Distribution account_no: 409000611074

In [ ]:
acc7_gauss = acc7.copy() 
scaler = StandardScaler()
acc7_gauss_scaled = pd.DataFrame(scaler.fit_transform(acc7_gauss[acc7.columns[2:]]), columns=acc7.columns[2:])
acc7_gauss_scaled.set_index(acc7_gauss.index, drop=True, inplace=True)
acc7_gauss_scaled.head(3)

In [ ]:
outliers_fraction = .01
model = EllipticEnvelope(contamination=outliers_fraction)
model.fit(acc7_gauss_scaled)

In [ ]:
acc7_gauss_scaled['anomaly_gauss'] = pd.Series(model.predict(acc7_gauss_scaled)).apply(lambda x: 0 if x==1 else 1).values
acc7_gauss['anomaly_gauss'] = acc7_gauss_scaled['anomaly_gauss']
acc7_gauss_scaled['anomaly_gauss'].value_counts()

In [ ]:
bank_main['gauss_7'] = 0
bank_main.loc[acc7_gauss_scaled.index,'gauss_7'] = acc7_gauss_scaled['anomaly_gauss']
bank_main['gauss_7'].value_counts()

In [ ]:
labels = acc7_gauss_scaled.anomaly_gauss
colors = {0:'blue', 1:'red'}

fig = plt.figure(1, figsize=(8,8))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(acc7_gauss_scaled.iloc[:,3], acc7_gauss_scaled.iloc[:,4], acc7_gauss_scaled.iloc[:,5], c=labels.apply(lambda x:colors[x]))
ax.set_title(f'GAUSSIAN DISTRIBUTION (account_no = {accounts[6]}, outliers_fraction = {100*outliers_fraction}%)', fontsize=14)
ax.set_xlabel('withdrawal_amt')
ax.set_ylabel('deposit_amt')
ax.set_zlabel('balance_amt')

In [ ]:
df = acc7_gauss.copy()
a= df.loc[df['anomaly_gauss']==1, ['date', 'balance_amt']]
plt.figure(figsize=(20,8))
plt.plot(df['date'], df['balance_amt'], color='blue', label='Normal')
plt.scatter(a['date'], a['balance_amt'], color='red', label='Anomaly')
plt.title(f'{accounts[6]} Gaussian Distribution Anomalies')
plt.xlabel('Date')
plt.ylabel('balance_amt')
plt.legend()
plt.show()

#### &emsp;(VIII)  Gaussian Distribution account_no: 409000493201

In [ ]:
acc8_gauss = acc8.copy() 
scaler = StandardScaler()
acc8_gauss_scaled = pd.DataFrame(scaler.fit_transform(acc8_gauss[acc5.columns[2:]]), columns=acc8.columns[2:])
acc8_gauss_scaled.set_index(acc8_gauss.index, drop=True, inplace=True)
acc8_gauss_scaled.head(3)

In [ ]:
outliers_fraction = .01
model = EllipticEnvelope(contamination=outliers_fraction)
model.fit(acc8_gauss_scaled)

In [ ]:
acc8_gauss_scaled['anomaly_gauss'] = pd.Series(model.predict(acc8_gauss_scaled)).apply(lambda x: 0 if x==1 else 1).values
acc8_gauss['anomaly_gauss'] = acc8_gauss_scaled['anomaly_gauss']
acc8_gauss_scaled['anomaly_gauss'].value_counts()

In [ ]:
bank_main['gauss_8'] = 0
bank_main.loc[acc8_gauss_scaled.index,'gauss_8'] = acc8_gauss_scaled['anomaly_gauss']
bank_main['gauss_8'].value_counts()

In [ ]:
labels = acc8_gauss_scaled.anomaly_gauss
colors = {0:'blue', 1:'red'}

fig = plt.figure(1, figsize=(8,8))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(acc8_gauss_scaled.iloc[:,3], acc8_gauss_scaled.iloc[:,4], acc8_gauss_scaled.iloc[:,5], c=labels.apply(lambda x:colors[x]))
ax.set_title(f'GAUSSIAN DISTRIBUTION (account_no = {accounts[7]}, outliers_fraction = {100*outliers_fraction}%)', fontsize=14)
ax.set_xlabel('withdrawal_amt')
ax.set_ylabel('deposit_amt')
ax.set_zlabel('balance_amt')

In [ ]:
df = acc8_gauss.copy()
a= df.loc[df['anomaly_gauss']==1, ['date', 'balance_amt']]
plt.figure(figsize=(20,8))
plt.plot(df['date'], df['balance_amt'], color='blue', label='Normal')
plt.scatter(a['date'], a['balance_amt'], color='red', label='Anomaly')
plt.title(f'{accounts[7]} Gaussian Distribution Anomalies')
plt.xlabel('Date')
plt.ylabel('balance_amt')
plt.legend()
plt.show()

#### &emsp;(IX)  Gaussian Distribution account_no: 409000425051

In [ ]:
acc9_gauss = acc9.copy() 
scaler = StandardScaler()
acc9_gauss_scaled = pd.DataFrame(scaler.fit_transform(acc9_gauss[acc9.columns[2:]]), columns=acc9.columns[2:])
acc9_gauss_scaled.set_index(acc9_gauss.index, drop=True, inplace=True)
acc9_gauss_scaled.head(3)

In [ ]:
outliers_fraction = .008
model = EllipticEnvelope(contamination=outliers_fraction)
model.fit(acc9_gauss_scaled)

In [ ]:
acc9_gauss_scaled['anomaly_gauss'] = pd.Series(model.predict(acc9_gauss_scaled)).apply(lambda x: 0 if x==1 else 1).values
acc9_gauss['anomaly_gauss'] = acc9_gauss_scaled['anomaly_gauss']
acc9_gauss_scaled['anomaly_gauss'].value_counts()

In [ ]:
bank_main['gauss_9'] = 0
bank_main.loc[acc9_gauss_scaled.index,'gauss_9'] = acc9_gauss_scaled['anomaly_gauss']
bank_main['gauss_9'].value_counts()

In [ ]:
labels = acc9_gauss_scaled.anomaly_gauss
colors = {0:'blue', 1:'red'}

fig = plt.figure(1, figsize=(8,8))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(acc9_gauss_scaled.iloc[:,3], acc9_gauss_scaled.iloc[:,4], acc9_gauss_scaled.iloc[:,5], c=labels.apply(lambda x:colors[x]))
ax.set_title(f'GAUSSIAN DISTRIBUTION (account_no = {accounts[8]}, outliers_fraction = {100*outliers_fraction}%)', fontsize=14)
ax.set_xlabel('withdrawal_amt')
ax.set_ylabel('deposit_amt')
ax.set_zlabel('balance_amt')

In [ ]:
df = acc9_gauss.copy()
a= df.loc[df['anomaly_gauss']==1, ['date', 'balance_amt']]
plt.figure(figsize=(20,8))
plt.plot(df['date'], df['balance_amt'], color='blue', label='Normal')
plt.scatter(a['date'], a['balance_amt'], color='red', label='Anomaly')
plt.title(f'{accounts[8]} Gaussian Distribution Anomalies')
plt.xlabel('Date')
plt.ylabel('balance_amt')
plt.legend()
plt.show()

#### &emsp;(X)  Gaussian Distribution account_no: 409000405747

In [ ]:
acc10_gauss = acc10.copy() 
scaler = StandardScaler()
acc10_gauss_scaled = pd.DataFrame(scaler.fit_transform(acc10_gauss[acc10.columns[2:]]), columns=acc10.columns[2:])
acc10_gauss_scaled.set_index(acc10_gauss.index, drop=True, inplace=True)
acc10_gauss_scaled.head(3)

In [ ]:
outliers_fraction = .08
model = EllipticEnvelope(contamination=outliers_fraction)
model.fit(acc10_gauss_scaled)

In [ ]:
acc10_gauss_scaled['anomaly_gauss'] = pd.Series(model.predict(acc10_gauss_scaled)).apply(lambda x: 0 if x==1 else 1).values
acc10_gauss['anomaly_gauss'] = acc10_gauss_scaled['anomaly_gauss']
acc10_gauss_scaled['anomaly_gauss'].value_counts()

In [ ]:
bank_main['gauss_10'] = 0
bank_main.loc[acc10_gauss_scaled.index,'gauss_10'] = acc10_gauss_scaled['anomaly_gauss']
bank_main['gauss_10'].value_counts()

In [ ]:
labels = acc10_gauss_scaled.anomaly_gauss
colors = {0:'blue', 1:'red'}

fig = plt.figure(1, figsize=(8,8))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(acc10_gauss_scaled.iloc[:,3], acc10_gauss_scaled.iloc[:,4], acc10_gauss_scaled.iloc[:,5], c=labels.apply(lambda x:colors[x]))
ax.set_title(f'GAUSSIAN DISTRIBUTION (account_no = {accounts[9]}, outliers_fraction = {100*outliers_fraction}%)', fontsize=14)
ax.set_xlabel('withdrawal_amt')
ax.set_ylabel('deposit_amt')
ax.set_zlabel('balance_amt')

In [ ]:
df = acc10_gauss.copy()
a= df.loc[df['anomaly_gauss']==1, ['date', 'balance_amt']]
plt.figure(figsize=(20,8))
plt.plot(df['date'], df['balance_amt'], color='blue', label='Normal')
plt.scatter(a['date'], a['balance_amt'], color='red', label='Anomaly')
plt.title(f'{accounts[9]} Gaussian Distribution Anomalies')
plt.xlabel('Date')
plt.ylabel('balance_amt')
plt.legend()
plt.show()

In [ ]:
bank_main['accounts_gaussian_distribution'] = bank_main.gauss_1 + bank_main.gauss_2 + bank_main.gauss_3 + bank_main.gauss_4 + bank_main.gauss_5 + bank_main.gauss_6 + bank_main.gauss_7 + bank_main.gauss_8 + bank_main.gauss_9 + bank_main.gauss_10

In [ ]:
bank_main['accounts_gaussian_distribution'].value_counts()

### iv. KMeans Clustering Anomaly Detection

#### &emsp;(I) Customer account_no_1196428

In [ ]:
acc1_kmeans = acc1.copy() 
scaler = StandardScaler()
acc1_kmeans_scaled = pd.DataFrame(scaler.fit_transform(acc1_kmeans[acc1.columns[2:]]), columns=acc1.columns[2:])
acc1_kmeans_scaled.set_index(acc1_kmeans.index, drop=True, inplace=True)
acc1_kmeans_scaled.head(3)

In [ ]:
# Instantiate the clustering model and visualizer
visualizer = KElbowVisualizer(KMeans(), k=(2,15))
plt.figure(figsize=(8,4)).patch.set_facecolor('xkcd:white')
visualizer.fit(acc1_kmeans_scaled)       # Fit the data to the visualizer
# visualizer.show()        # Finalize and render the figure

In [ ]:
kmeans = KMeans(n_clusters=4)
kmeans.fit(acc1_kmeans_scaled)
labels = kmeans.labels_

In [ ]:
fig = plt.figure(1, figsize=(6,6))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(acc1_kmeans_scaled.iloc[:,3], acc1_kmeans_scaled.iloc[:,4], acc1_kmeans_scaled.iloc[:,5], c=labels.astype(np.float), cmap='rainbow')
ax.set_title(f'K MEANS {accounts[0]}', fontsize=14)
ax.set_xlabel('withdrawal_amt')
ax.set_ylabel('deposit_amt')
ax.set_zlabel('balance_amt')

In [ ]:
x = acc1_kmeans_scaled
pca = PCA()
pipeline = make_pipeline(pca)
pipeline.fit(x)
# Plot the principal components against their inertia
features = range(pca.n_components_)
_ = plt.figure(figsize=(15, 5))
_ = plt.bar(features, pca.explained_variance_)
_ = plt.xlabel('PCA feature')
_ = plt.ylabel('Variance')
_ = plt.xticks(features)
_ = plt.title(f"account_no{accounts[0]} Importance of the Principal Components based on inertia")
plt.show()

In [ ]:
acc1_pca = pd.DataFrame(PCA(n_components=5).fit_transform(acc1_kmeans_scaled), columns=['c1','c2', 'c3', 'c4', 'c5'])

In [ ]:
kmeans = KMeans(n_clusters=4)
kmeans.fit(acc1_pca)
labels = kmeans.predict(acc1_pca)
unique_elements, counts_elements = np.unique(labels, return_counts=True)
clusters = np.asarray((unique_elements, counts_elements))

# Assume that 0.5% of the entire data set are anomalies 
outliers_fraction = 0.003
# get the distance between each point and its nearest centroid. The biggest distances are considered as anomaly
distance = getDistanceByPoint(acc1_pca, kmeans)
# number of observations that equate to the 0.5%  of the entire data set
number_of_outliers = int(outliers_fraction*len(distance))
# Take the minimum of the largest 0.5%  of the distances as the threshold
threshold = distance.nlargest(number_of_outliers).min()
# anomaly1 contain the anomaly result of the above method Cluster (0:normal, 1:anomaly) 
acc1_pca['anomaly_kmeans'] = (distance >= threshold).astype(int)

In [ ]:
acc1_pca.index = acc1.index
acc1_pca.anomaly_kmeans.value_counts()

In [ ]:
acc1_kmeans_scaled['anomaly_kmeans'] = acc1_pca.anomaly_kmeans
bank_main['km_1'] = 0
bank_main.loc[acc1_kmeans_scaled.index,'km_1'] = acc1_kmeans_scaled['anomaly_kmeans']
bank_main['km_1'].value_counts()

In [ ]:
labels = acc1_kmeans_scaled.anomaly_kmeans
colors = {0:'blue', 1:'red'}

fig = plt.figure(1, figsize=(8,8))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(acc1_kmeans_scaled.iloc[:,3], acc1_kmeans_scaled.iloc[:,4], acc1_kmeans_scaled.iloc[:,5], c=labels.apply(lambda x:colors[x]))
ax.set_title(f'KMEANS Anomalies account_no = {accounts[0]} (outliers_fraction = {100*outliers_fraction})%', fontsize=14)
ax.set_xlabel('withdrawal_amt')
ax.set_ylabel('deposit_amt')
ax.set_zlabel('balance_amt')

In [ ]:
acc1_kmeans['anomaly_kmeans'] = acc1_kmeans_scaled['anomaly_kmeans']
df = acc1_kmeans.copy()
a= df.loc[df['anomaly_kmeans']==1, ['date', 'balance_amt']]
plt.figure(figsize=(20,8))
plt.plot(df['date'], df['balance_amt'], color='blue', label='Normal')
plt.scatter(a['date'], a['balance_amt'], color='red', label='Anomaly')
plt.title(f'{accounts[0]} Kmeans Distribution Anomalies')
plt.xlabel('Date')
plt.ylabel('balance_amt')
plt.legend()
plt.show()

#### &emsp;(II) Customer account_no_409000362497

In [ ]:
acc2_kmeans = acc2.copy() 
scaler = StandardScaler()
acc2_kmeans_scaled = pd.DataFrame(scaler.fit_transform(acc2_kmeans[acc2.columns[2:]]), columns=acc2.columns[2:])
acc2_kmeans_scaled.set_index(acc2_kmeans.index, drop=True, inplace=True)
acc2_kmeans_scaled.head(3)

In [ ]:
# Instantiate the clustering model and visualizer
visualizer = KElbowVisualizer(KMeans(), k=(2,15))
plt.figure(figsize=(8,4)).patch.set_facecolor('xkcd:white')
visualizer.fit(acc2_kmeans_scaled)       # Fit the data to the visualizer
# visualizer.show()        # Finalize and render the figure

In [ ]:
kmeans = KMeans(n_clusters=4)
kmeans.fit(acc2_kmeans_scaled)
labels = kmeans.labels_

In [ ]:
fig = plt.figure(1, figsize=(6,6))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(acc2_kmeans_scaled.iloc[:,3], acc2_kmeans_scaled.iloc[:,4], acc2_kmeans_scaled.iloc[:,5], c=labels.astype(np.float), cmap='rainbow')
ax.set_title(f'K MEANS {accounts[1]}', fontsize=14)
ax.set_xlabel('withdrawal_amt')
ax.set_ylabel('deposit_amt')
ax.set_zlabel('balance_amt')

In [ ]:
x = acc2_kmeans_scaled
pca = PCA()
pipeline = make_pipeline(pca)
pipeline.fit(x)
# Plot the principal components against their inertia
features = range(pca.n_components_)
_ = plt.figure(figsize=(15, 5))
_ = plt.bar(features, pca.explained_variance_)
_ = plt.xlabel('PCA feature')
_ = plt.ylabel('Variance')
_ = plt.xticks(features)
_ = plt.title(f"account_no{accounts[0]} Importance of the Principal Components based on inertia")
plt.show()

In [ ]:
acc2_pca = pd.DataFrame(PCA(n_components=6).fit_transform(acc2_kmeans_scaled), columns=['c1','c2', 'c3', 'c4', 'c5', 'c6'])

In [ ]:
kmeans = KMeans(n_clusters=4)
kmeans.fit(acc2_pca)
labels = kmeans.predict(acc2_pca)
unique_elements, counts_elements = np.unique(labels, return_counts=True)
clusters = np.asarray((unique_elements, counts_elements))

outliers_fraction = 0.003
distance = getDistanceByPoint(acc2_pca, kmeans)
number_of_outliers = int(outliers_fraction*len(distance))
threshold = distance.nlargest(number_of_outliers).min()
acc2_pca['anomaly_kmeans'] = (distance >= threshold).astype(int)

In [ ]:
acc2_pca.index = acc2.index
acc2_pca.anomaly_kmeans.value_counts()

In [ ]:
acc2_kmeans_scaled['anomaly_kmeans'] = acc2_pca.anomaly_kmeans
bank_main['km_2'] = 0
bank_main.loc[acc2_kmeans_scaled.index,'km_2'] = acc2_kmeans_scaled['anomaly_kmeans']
bank_main['km_2'].value_counts()

In [ ]:
labels = acc2_kmeans_scaled.anomaly_kmeans
colors = {0:'blue', 1:'red'}

fig = plt.figure(1, figsize=(8,8))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(acc2_kmeans_scaled.iloc[:,3], acc2_kmeans_scaled.iloc[:,4], acc2_kmeans_scaled.iloc[:,5], c=labels.apply(lambda x:colors[x]))
ax.set_title(f'KMEANS Anomalies account_no = {accounts[1]} (outliers_fraction = {100*outliers_fraction})%', fontsize=14)
ax.set_xlabel('withdrawal_amt')
ax.set_ylabel('deposit_amt')
ax.set_zlabel('balance_amt')

In [ ]:
acc2_kmeans['anomaly_kmeans'] = acc2_kmeans_scaled['anomaly_kmeans']
df = acc2_kmeans.copy()
a= df.loc[df['anomaly_kmeans']==1, ['date', 'balance_amt']]
plt.figure(figsize=(20,8))
plt.plot(df['date'], df['balance_amt'], color='blue', label='Normal')
plt.scatter(a['date'], a['balance_amt'], color='red', label='Anomaly')
plt.title(f'{accounts[1]} Kmeans Distribution Anomalies')
plt.xlabel('Date')
plt.ylabel('balance_amt')
plt.legend()
plt.show()

In [ ]:
accounts[2]

#### &emsp;(III) Customer account_no_409000438620

In [ ]:
acc3_kmeans = acc3.copy() 
scaler = StandardScaler()
acc3_kmeans_scaled = pd.DataFrame(scaler.fit_transform(acc3_kmeans[acc3.columns[2:]]), columns=acc3.columns[2:])
acc3_kmeans_scaled.set_index(acc3_kmeans.index, drop=True, inplace=True)
acc3_kmeans_scaled.head(3)

In [ ]:
# Instantiate the clustering model and visualizer
visualizer = KElbowVisualizer(KMeans(), k=(2,15))
plt.figure(figsize=(8,4)).patch.set_facecolor('xkcd:white')
visualizer.fit(acc3_kmeans_scaled)       # Fit the data to the visualizer
# visualizer.show()        # Finalize and render the figure

In [ ]:
kmeans = KMeans(n_clusters=6)
kmeans.fit(acc3_kmeans_scaled)
labels = kmeans.labels_

In [ ]:
fig = plt.figure(1, figsize=(6,6))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(acc3_kmeans_scaled.iloc[:,3], acc3_kmeans_scaled.iloc[:,4], acc3_kmeans_scaled.iloc[:,5], c=labels.astype(np.float), cmap='rainbow')
ax.set_title(f'K MEANS {accounts[2]}', fontsize=14)
ax.set_xlabel('withdrawal_amt')
ax.set_ylabel('deposit_amt')
ax.set_zlabel('balance_amt')

In [ ]:
x = acc3_kmeans_scaled
pca = PCA()
pipeline = make_pipeline(pca)
pipeline.fit(x)
# Plot the principal components against their inertia
features = range(pca.n_components_)
_ = plt.figure(figsize=(15, 5))
_ = plt.bar(features, pca.explained_variance_)
_ = plt.xlabel('PCA feature')
_ = plt.ylabel('Variance')
_ = plt.xticks(features)
_ = plt.title(f"account_no{accounts[2]} Importance of the Principal Components based on inertia")
plt.show()

In [ ]:
acc3_pca = pd.DataFrame(PCA(n_components=7).fit_transform(acc3_kmeans_scaled), columns=['c1','c2', 'c3', 'c4', 'c5', 'c6', 'c7'])

In [ ]:
kmeans = KMeans(n_clusters=6)
kmeans.fit(acc3_pca)
labels = kmeans.predict(acc3_pca)

unique_elements, counts_elements = np.unique(labels, return_counts=True)
clusters = np.asarray((unique_elements, counts_elements))

outliers_fraction = 0.003
distance = getDistanceByPoint(acc3_pca, kmeans)
number_of_outliers = int(outliers_fraction*len(distance))
threshold = distance.nlargest(number_of_outliers).min()
acc3_pca['anomaly_kmeans'] = (distance >= threshold).astype(int)

In [ ]:
acc3_pca.index = acc3.index
acc3_pca.anomaly_kmeans.value_counts()

In [ ]:
acc3_kmeans_scaled['anomaly_kmeans'] = acc3_pca.anomaly_kmeans
bank_main['km_3'] = 0
bank_main.loc[acc3_kmeans_scaled.index,'km_3'] = acc3_kmeans_scaled['anomaly_kmeans']
bank_main['km_3'].value_counts()

In [ ]:
labels = acc3_kmeans_scaled.anomaly_kmeans
colors = {0:'blue', 1:'red'}

fig = plt.figure(1, figsize=(8,8))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(acc3_kmeans_scaled.iloc[:,3], acc3_kmeans_scaled.iloc[:,4], acc3_kmeans_scaled.iloc[:,5], c=labels.apply(lambda x:colors[x]))
ax.set_title(f'KMEANS Anomalies account_no = {accounts[2]} (outliers_fraction = {100*outliers_fraction})%', fontsize=14)
ax.set_xlabel('withdrawal_amt')
ax.set_ylabel('deposit_amt')
ax.set_zlabel('balance_amt')

In [ ]:
acc3_kmeans['anomaly_kmeans'] = acc3_kmeans_scaled['anomaly_kmeans']
df = acc3_kmeans.copy()
a= df.loc[df['anomaly_kmeans']==1, ['date', 'balance_amt']]
plt.figure(figsize=(20,8))
plt.plot(df['date'], df['balance_amt'], color='blue', label='Normal')
plt.scatter(a['date'], a['balance_amt'], color='red', label='Anomaly')
plt.title(f'{accounts[2]} Kmeans Distribution Anomalies')
plt.xlabel('Date')
plt.ylabel('balance_amt')
plt.legend()
plt.show()

In [ ]:
accounts[3]

#### &emsp;(IV) Customer account_no_1196711

In [ ]:
acc4_kmeans = acc4.copy() 
scaler = StandardScaler()
acc4_kmeans_scaled = pd.DataFrame(scaler.fit_transform(acc4_kmeans[acc4.columns[2:]]), columns=acc4.columns[2:])
acc4_kmeans_scaled.set_index(acc4_kmeans.index, drop=True, inplace=True)
acc4_kmeans_scaled.head(3)

In [ ]:
# Instantiate the clustering model and visualizer
visualizer = KElbowVisualizer(KMeans(), k=(2,15))
plt.figure(figsize=(8,4)).patch.set_facecolor('xkcd:white')
visualizer.fit(acc4_kmeans_scaled)       # Fit the data to the visualizer
# visualizer.show()        # Finalize and render the figure

In [ ]:
kmeans = KMeans(n_clusters=9)
kmeans.fit(acc4_kmeans_scaled)
labels = kmeans.labels_

In [ ]:
fig = plt.figure(1, figsize=(6,6))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(acc4_kmeans_scaled.iloc[:,3], acc4_kmeans_scaled.iloc[:,4], acc4_kmeans_scaled.iloc[:,5], c=labels.astype(np.float), cmap='rainbow')
ax.set_title(f'K MEANS {accounts[3]}', fontsize=14)
ax.set_xlabel('withdrawal_amt')
ax.set_ylabel('deposit_amt')
ax.set_zlabel('balance_amt')

In [ ]:
x = acc4_kmeans_scaled
pca = PCA()
pipeline = make_pipeline(pca)
pipeline.fit(x)
# Plot the principal components against their inertia
features = range(pca.n_components_)
_ = plt.figure(figsize=(15, 5))
_ = plt.bar(features, pca.explained_variance_)
_ = plt.xlabel('PCA feature')
_ = plt.ylabel('Variance')
_ = plt.xticks(features)
_ = plt.title(f"account_no{accounts[3]} Importance of the Principal Components based on inertia")
plt.show()

In [ ]:
acc4_pca = pd.DataFrame(PCA(n_components=7).fit_transform(acc4_kmeans_scaled), columns=['c1','c2', 'c3', 'c4', 'c5', 'c6', 'c7'])

In [ ]:
kmeans = KMeans(n_clusters=9)
kmeans.fit(acc4_pca)
labels = kmeans.predict(acc4_pca)

unique_elements, counts_elements = np.unique(labels, return_counts=True)
clusters = np.asarray((unique_elements, counts_elements))

outliers_fraction = 0.005
distance = getDistanceByPoint(acc4_pca, kmeans)

number_of_outliers = int(outliers_fraction*len(distance))
threshold = distance.nlargest(number_of_outliers).min()
acc4_pca['anomaly_kmeans'] = (distance >= threshold).astype(int)

In [ ]:
acc4_pca.index = acc4.index
acc4_pca.anomaly_kmeans.value_counts()

In [ ]:
acc4_kmeans_scaled['anomaly_kmeans'] = acc4_pca.anomaly_kmeans
bank_main['km_4'] = 0
bank_main.loc[acc4_kmeans_scaled.index,'km_4'] = acc4_kmeans_scaled['anomaly_kmeans']
bank_main['km_4'].value_counts()

In [ ]:
labels = acc4_kmeans_scaled.anomaly_kmeans
colors = {0:'blue', 1:'red'}

fig = plt.figure(1, figsize=(8,8))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(acc4_kmeans_scaled.iloc[:,3], acc4_kmeans_scaled.iloc[:,4], acc4_kmeans_scaled.iloc[:,5], c=labels.apply(lambda x:colors[x]))
ax.set_title(f'KMEANS Anomalies account_no = {accounts[3]} (outliers_fraction = {100*outliers_fraction})%', fontsize=14)
ax.set_xlabel('withdrawal_amt')
ax.set_ylabel('deposit_amt')
ax.set_zlabel('balance_amt')

In [ ]:
acc4_kmeans['anomaly_kmeans'] = acc4_kmeans_scaled['anomaly_kmeans']
df = acc4_kmeans.copy()
a= df.loc[df['anomaly_kmeans']==1, ['date', 'balance_amt']]
plt.figure(figsize=(20,8))
plt.plot(df['date'], df['balance_amt'], color='blue', label='Normal')
plt.scatter(a['date'], a['balance_amt'], color='red', label='Anomaly')
plt.title(f'{accounts[3]} Kmeans Distribution Anomalies')
plt.xlabel('Date')
plt.ylabel('balance_amt')
plt.legend()
plt.show()

In [ ]:
accounts[4]

#### &emsp;(V) Customer account_no_409000493210

In [ ]:
acc5_kmeans = acc5.copy() 
scaler = StandardScaler()
acc5_kmeans_scaled = pd.DataFrame(scaler.fit_transform(acc5_kmeans[acc5.columns[2:]]), columns=acc5.columns[2:])
acc5_kmeans_scaled.set_index(acc5_kmeans.index, drop=True, inplace=True)
acc5_kmeans_scaled.head(3)

In [ ]:
# Instantiate the clustering model and visualizer
visualizer = KElbowVisualizer(KMeans(), k=(2,15))
plt.figure(figsize=(8,4)).patch.set_facecolor('xkcd:white')
visualizer.fit(acc5_kmeans_scaled)       # Fit the data to the visualizer
# visualizer.show()        # Finalize and render the figure

In [ ]:
kmeans = KMeans(n_clusters=4)
kmeans.fit(acc5_kmeans_scaled)
labels = kmeans.labels_

In [ ]:
fig = plt.figure(1, figsize=(6,6))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(acc5_kmeans_scaled.iloc[:,3], acc5_kmeans_scaled.iloc[:,4], acc5_kmeans_scaled.iloc[:,5], c=labels.astype(np.float), cmap='rainbow')
ax.set_title(f'K MEANS {accounts[4]}', fontsize=14)
ax.set_xlabel('withdrawal_amt')
ax.set_ylabel('deposit_amt')
ax.set_zlabel('balance_amt')

In [ ]:
x = acc5_kmeans_scaled
pca = PCA()
pipeline = make_pipeline(pca)
pipeline.fit(x)
# Plot the principal components against their inertia
features = range(pca.n_components_)
_ = plt.figure(figsize=(15, 5))
_ = plt.bar(features, pca.explained_variance_)
_ = plt.xlabel('PCA feature')
_ = plt.ylabel('Variance')
_ = plt.xticks(features)
_ = plt.title(f"account_no{accounts[4]} Importance of the Principal Components based on inertia")
plt.show()

In [ ]:
acc5_pca = pd.DataFrame(PCA(n_components=3).fit_transform(acc5_kmeans_scaled), columns=['c1','c2', 'c3'])

In [ ]:
kmeans = KMeans(n_clusters=4)
kmeans.fit(acc5_pca)
labels = kmeans.predict(acc5_pca)

unique_elements, counts_elements = np.unique(labels, return_counts=True)
clusters = np.asarray((unique_elements, counts_elements))

outliers_fraction = 0.005
distance = getDistanceByPoint(acc5_pca, kmeans)

number_of_outliers = int(outliers_fraction*len(distance))
threshold = distance.nlargest(number_of_outliers).min()
acc5_pca['anomaly_kmeans'] = (distance >= threshold).astype(int)

In [ ]:
acc5_pca.index = acc5.index
acc5_pca.anomaly_kmeans.value_counts()

In [ ]:
acc5_kmeans_scaled['anomaly_kmeans'] = acc5_pca.anomaly_kmeans
bank_main['km_5'] = 0
bank_main.loc[acc5_kmeans_scaled.index,'km_5'] = acc5_kmeans_scaled['anomaly_kmeans']
bank_main['km_5'].value_counts()

In [ ]:
labels = acc5_kmeans_scaled.anomaly_kmeans
colors = {0:'blue', 1:'red'}

fig = plt.figure(1, figsize=(8,8))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(acc5_kmeans_scaled.iloc[:,3], acc5_kmeans_scaled.iloc[:,4], acc5_kmeans_scaled.iloc[:,5], c=labels.apply(lambda x:colors[x]))
ax.set_title(f'KMEANS Anomalies account_no = {accounts[4]} (outliers_fraction = {100*outliers_fraction})%', fontsize=14)
ax.set_xlabel('withdrawal_amt')
ax.set_ylabel('deposit_amt')
ax.set_zlabel('balance_amt')

In [ ]:
acc5_kmeans['anomaly_kmeans'] = acc5_kmeans_scaled['anomaly_kmeans']
df = acc5_kmeans.copy()
a= df.loc[df['anomaly_kmeans']==1, ['date', 'balance_amt']]
plt.figure(figsize=(20,8))
plt.plot(df['date'], df['balance_amt'], color='blue', label='Normal')
plt.scatter(a['date'], a['balance_amt'], color='red', label='Anomaly')
plt.title(f'{accounts[4]} Kmeans Distribution Anomalies')
plt.xlabel('Date')
plt.ylabel('balance_amt')
plt.legend()
plt.show()

#### &emsp;(V) Customer account_no_409000438611

In [ ]:
acc6_kmeans = acc6.copy() 
scaler = StandardScaler()
acc6_kmeans_scaled = pd.DataFrame(scaler.fit_transform(acc6_kmeans[acc6.columns[2:]]), columns=acc6.columns[2:])
acc6_kmeans_scaled.set_index(acc6_kmeans.index, drop=True, inplace=True)
acc6_kmeans_scaled.head(3)

In [ ]:
# Instantiate the clustering model and visualizer
visualizer = KElbowVisualizer(KMeans(), k=(2,15))
plt.figure(figsize=(8,4)).patch.set_facecolor('xkcd:white')
visualizer.fit(acc6_kmeans_scaled)       # Fit the data to the visualizer
# visualizer.show()        # Finalize and render the figure

In [ ]:
kmeans = KMeans(n_clusters=6)
kmeans.fit(acc6_kmeans_scaled)
labels = kmeans.labels_

In [ ]:
fig = plt.figure(1, figsize=(6,6))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(acc6_kmeans_scaled.iloc[:,3], acc6_kmeans_scaled.iloc[:,4], acc6_kmeans_scaled.iloc[:,5], c=labels.astype(np.float), cmap='rainbow')
ax.set_title(f'K MEANS {accounts[5]}', fontsize=14)
ax.set_xlabel('withdrawal_amt')
ax.set_ylabel('deposit_amt')
ax.set_zlabel('balance_amt')

In [ ]:
x = acc6_kmeans_scaled
pca = PCA()
pipeline = make_pipeline(pca)
pipeline.fit(x)
# Plot the principal components against their inertia
features = range(pca.n_components_)
_ = plt.figure(figsize=(15, 5))
_ = plt.bar(features, pca.explained_variance_)
_ = plt.xlabel('PCA feature')
_ = plt.ylabel('Variance')
_ = plt.xticks(features)
_ = plt.title(f"account_no{accounts[5]} Importance of the Principal Components based on inertia")
plt.show()

In [ ]:
acc6_pca = pd.DataFrame(PCA(n_components=5).fit_transform(acc6_kmeans_scaled), columns=['c1','c2', 'c3', 'c4', 'c5'])

In [ ]:
kmeans = KMeans(n_clusters=6)
kmeans.fit(acc6_pca)
labels = kmeans.predict(acc6_pca)

unique_elements, counts_elements = np.unique(labels, return_counts=True)
clusters = np.asarray((unique_elements, counts_elements))

outliers_fraction = 0.005
distance = getDistanceByPoint(acc6_pca, kmeans)

number_of_outliers = int(outliers_fraction*len(distance))
threshold = distance.nlargest(number_of_outliers).min()
acc6_pca['anomaly_kmeans'] = (distance >= threshold).astype(int)

In [ ]:
acc6_pca.index = acc6.index
acc6_pca.anomaly_kmeans.value_counts()

In [ ]:
acc6_kmeans_scaled['anomaly_kmeans'] = acc6_pca.anomaly_kmeans
bank_main['km_6'] = 0
bank_main.loc[acc6_kmeans_scaled.index,'km_6'] = acc6_kmeans_scaled['anomaly_kmeans']
bank_main['km_6'].value_counts()

In [ ]:
labels = acc6_kmeans_scaled.anomaly_kmeans
colors = {0:'blue', 1:'red'}

fig = plt.figure(1, figsize=(8,8))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(acc6_kmeans_scaled.iloc[:,3], acc6_kmeans_scaled.iloc[:,4], acc6_kmeans_scaled.iloc[:,5], c=labels.apply(lambda x:colors[x]))
ax.set_title(f'KMEANS Anomalies account_no = {accounts[5]} (outliers_fraction = {100*outliers_fraction})%', fontsize=14)
ax.set_xlabel('withdrawal_amt')
ax.set_ylabel('deposit_amt')
ax.set_zlabel('balance_amt')

In [ ]:
acc6_kmeans['anomaly_kmeans'] = acc6_kmeans_scaled['anomaly_kmeans']
df = acc6_kmeans.copy()
a= df.loc[df['anomaly_kmeans']==1, ['date', 'balance_amt']]
plt.figure(figsize=(20,8))
plt.plot(df['date'], df['balance_amt'], color='blue', label='Normal')
plt.scatter(a['date'], a['balance_amt'], color='red', label='Anomaly')
plt.title(f'{accounts[5]} Kmeans Distribution Anomalies')
plt.xlabel('Date')
plt.ylabel('balance_amt')
plt.legend()
plt.show()

#### &emsp;(VII) Customer account_no_409000611074

In [ ]:
acc7_kmeans = acc7.copy() 
scaler = StandardScaler()
acc7_kmeans_scaled = pd.DataFrame(scaler.fit_transform(acc7_kmeans[acc7.columns[2:]]), columns=acc7.columns[2:])
acc7_kmeans_scaled.set_index(acc7_kmeans.index, drop=True, inplace=True)
acc7_kmeans_scaled.head(3)

In [ ]:
# Instantiate the clustering model and visualizer
visualizer = KElbowVisualizer(KMeans(), k=(2,15))
plt.figure(figsize=(8,4)).patch.set_facecolor('xkcd:white')
visualizer.fit(acc7_kmeans_scaled)       # Fit the data to the visualizer
# visualizer.show()        # Finalize and render the figure

In [ ]:
kmeans = KMeans(n_clusters=5)
kmeans.fit(acc7_kmeans_scaled)
labels = kmeans.labels_

In [ ]:
fig = plt.figure(1, figsize=(6,6))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(acc7_kmeans_scaled.iloc[:,3], acc7_kmeans_scaled.iloc[:,4], acc7_kmeans_scaled.iloc[:,5], c=labels.astype(np.float), cmap='rainbow')
ax.set_title(f'K MEANS {accounts[6]}', fontsize=14)
ax.set_xlabel('withdrawal_amt')
ax.set_ylabel('deposit_amt')
ax.set_zlabel('balance_amt')

In [ ]:
x = acc7_kmeans_scaled
pca = PCA()
pipeline = make_pipeline(pca)
pipeline.fit(x)
# Plot the principal components against their inertia
features = range(pca.n_components_)
_ = plt.figure(figsize=(15, 5))
_ = plt.bar(features, pca.explained_variance_)
_ = plt.xlabel('PCA feature')
_ = plt.ylabel('Variance')
_ = plt.xticks(features)
_ = plt.title(f"account_no{accounts[6]} Importance of the Principal Components based on inertia")
plt.show()

In [ ]:
acc7_pca = pd.DataFrame(PCA(n_components=5).fit_transform(acc7_kmeans_scaled), columns=['c1','c2', 'c3', 'c4', 'c5'])

In [ ]:
kmeans = KMeans(n_clusters=5)
kmeans.fit(acc7_pca)
labels = kmeans.predict(acc7_pca)

unique_elements, counts_elements = np.unique(labels, return_counts=True)
clusters = np.asarray((unique_elements, counts_elements))

outliers_fraction = 0.01
distance = getDistanceByPoint(acc7_pca, kmeans)

number_of_outliers = int(outliers_fraction*len(distance))
threshold = distance.nlargest(number_of_outliers).min()
acc7_pca['anomaly_kmeans'] = (distance >= threshold).astype(int)

In [ ]:
acc7_pca.index = acc7.index
acc7_pca.anomaly_kmeans.value_counts()

In [ ]:
acc7_kmeans_scaled['anomaly_kmeans'] = acc7_pca.anomaly_kmeans
bank_main['km_7'] = 0
bank_main.loc[acc7_kmeans_scaled.index,'km_7'] = acc7_kmeans_scaled['anomaly_kmeans']
bank_main['km_7'].value_counts()

In [ ]:
labels = acc7_kmeans_scaled.anomaly_kmeans
colors = {0:'blue', 1:'red'}

fig = plt.figure(1, figsize=(8,8))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(acc7_kmeans_scaled.iloc[:,3], acc7_kmeans_scaled.iloc[:,4], acc7_kmeans_scaled.iloc[:,5], c=labels.apply(lambda x:colors[x]))
ax.set_title(f'KMEANS Anomalies account_no = {accounts[6]} (outliers_fraction = {100*outliers_fraction})%', fontsize=14)
ax.set_xlabel('withdrawal_amt')
ax.set_ylabel('deposit_amt')
ax.set_zlabel('balance_amt')

In [ ]:
acc7_kmeans['anomaly_kmeans'] = acc7_kmeans_scaled['anomaly_kmeans']
df = acc7_kmeans.copy()
a= df.loc[df['anomaly_kmeans']==1, ['date', 'balance_amt']]
plt.figure(figsize=(20,8))
plt.plot(df['date'], df['balance_amt'], color='blue', label='Normal')
plt.scatter(a['date'], a['balance_amt'], color='red', label='Anomaly')
plt.title(f'{accounts[6]} Kmeans Distribution Anomalies')
plt.xlabel('Date')
plt.ylabel('balance_amt')
plt.legend()
plt.show()

#### &emsp;(VIII) Customer account_no_409000493201

In [ ]:
acc8_kmeans = acc8.copy() 
scaler = StandardScaler()
acc8_kmeans_scaled = pd.DataFrame(scaler.fit_transform(acc8_kmeans[acc8.columns[2:]]), columns=acc8.columns[2:])
acc8_kmeans_scaled.set_index(acc8_kmeans.index, drop=True, inplace=True)
acc8_kmeans_scaled.head(3)

In [ ]:
# Instantiate the clustering model and visualizer
visualizer = KElbowVisualizer(KMeans(), k=(2,15))
plt.figure(figsize=(8,4)).patch.set_facecolor('xkcd:white')
visualizer.fit(acc8_kmeans_scaled)       # Fit the data to the visualizer
# visualizer.show()        # Finalize and render the figure

In [ ]:
kmeans = KMeans(n_clusters=5)
kmeans.fit(acc8_kmeans_scaled)
labels = kmeans.labels_

In [ ]:
fig = plt.figure(1, figsize=(6,6))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(acc8_kmeans_scaled.iloc[:,3], acc8_kmeans_scaled.iloc[:,4], acc8_kmeans_scaled.iloc[:,5], c=labels.astype(np.float), cmap='rainbow')
ax.set_title(f'K MEANS {accounts[7]}', fontsize=14)
ax.set_xlabel('withdrawal_amt')
ax.set_ylabel('deposit_amt')
ax.set_zlabel('balance_amt')

In [ ]:
x = acc8_kmeans_scaled
pca = PCA()
pipeline = make_pipeline(pca)
pipeline.fit(x)
# Plot the principal components against their inertia
features = range(pca.n_components_)
_ = plt.figure(figsize=(15, 5))
_ = plt.bar(features, pca.explained_variance_)
_ = plt.xlabel('PCA feature')
_ = plt.ylabel('Variance')
_ = plt.xticks(features)
_ = plt.title(f"account_no{accounts[7]} Importance of the Principal Components based on inertia")
plt.show()

In [ ]:
acc8_pca = pd.DataFrame(PCA(n_components=5).fit_transform(acc8_kmeans_scaled), columns=['c1','c2', 'c3', 'c4', 'c5'])

In [ ]:
kmeans = KMeans(n_clusters=5)
kmeans.fit(acc8_pca)
labels = kmeans.predict(acc8_pca)

unique_elements, counts_elements = np.unique(labels, return_counts=True)
clusters = np.asarray((unique_elements, counts_elements))

outliers_fraction = 0.007
distance = getDistanceByPoint(acc8_pca, kmeans)

number_of_outliers = int(outliers_fraction*len(distance))
threshold = distance.nlargest(number_of_outliers).min()
acc8_pca['anomaly_kmeans'] = (distance >= threshold).astype(int)

In [ ]:
acc8_pca.index = acc8.index
acc8_pca.anomaly_kmeans.value_counts()

In [ ]:
acc8_kmeans_scaled['anomaly_kmeans'] = acc8_pca.anomaly_kmeans
bank_main['km_8'] = 0
bank_main.loc[acc8_kmeans_scaled.index,'km_8'] = acc8_kmeans_scaled['anomaly_kmeans']
bank_main['km_8'].value_counts()

In [ ]:
labels = acc8_kmeans_scaled.anomaly_kmeans
colors = {0:'blue', 1:'red'}

fig = plt.figure(1, figsize=(8,8))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(acc8_kmeans_scaled.iloc[:,3], acc8_kmeans_scaled.iloc[:,4], acc8_kmeans_scaled.iloc[:,5], c=labels.apply(lambda x:colors[x]))
ax.set_title(f'KMEANS Anomalies account_no = {accounts[7]} (outliers_fraction = {100*outliers_fraction})%', fontsize=14)
ax.set_xlabel('withdrawal_amt')
ax.set_ylabel('deposit_amt')
ax.set_zlabel('balance_amt')

In [ ]:
acc8_kmeans['anomaly_kmeans'] = acc8_kmeans_scaled['anomaly_kmeans']
df = acc8_kmeans.copy()
a= df.loc[df['anomaly_kmeans']==1, ['date', 'balance_amt']]
plt.figure(figsize=(20,8))
plt.plot(df['date'], df['balance_amt'], color='blue', label='Normal')
plt.scatter(a['date'], a['balance_amt'], color='red', label='Anomaly')
plt.title(f'{accounts[7]} Kmeans Distribution Anomalies')
plt.xlabel('Date')
plt.ylabel('balance_amt')
plt.legend()
plt.show()

#### &emsp;(IX) Customer account_no_409000425051

In [ ]:
acc9_kmeans = acc9.copy() 
scaler = StandardScaler()
acc9_kmeans_scaled = pd.DataFrame(scaler.fit_transform(acc9_kmeans[acc9.columns[2:]]), columns=acc9.columns[2:])
acc9_kmeans_scaled.set_index(acc9_kmeans.index, drop=True, inplace=True)
acc9_kmeans_scaled.head(3)

In [ ]:
# Instantiate the clustering model and visualizer
visualizer = KElbowVisualizer(KMeans(), k=(2,15))
plt.figure(figsize=(8,4)).patch.set_facecolor('xkcd:white')
visualizer.fit(acc9_kmeans_scaled)       # Fit the data to the visualizer
# visualizer.show()        # Finalize and render the figure

In [ ]:
kmeans = KMeans(n_clusters=7)
kmeans.fit(acc9_kmeans_scaled)
labels = kmeans.labels_

In [ ]:
fig = plt.figure(1, figsize=(6,6))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(acc9_kmeans_scaled.iloc[:,3], acc9_kmeans_scaled.iloc[:,4], acc9_kmeans_scaled.iloc[:,5], c=labels.astype(np.float), cmap='rainbow')
ax.set_title(f'K MEANS {accounts[8]}', fontsize=14)
ax.set_xlabel('withdrawal_amt')
ax.set_ylabel('deposit_amt')
ax.set_zlabel('balance_amt')

In [ ]:
x = acc9_kmeans_scaled
pca = PCA()
pipeline = make_pipeline(pca)
pipeline.fit(x)
# Plot the principal components against their inertia
features = range(pca.n_components_)
_ = plt.figure(figsize=(15, 5))
_ = plt.bar(features, pca.explained_variance_)
_ = plt.xlabel('PCA feature')
_ = plt.ylabel('Variance')
_ = plt.xticks(features)
_ = plt.title(f"account_no{accounts[8]} Importance of the Principal Components based on inertia")
plt.show()

In [ ]:
acc9_pca = pd.DataFrame(PCA(n_components=6).fit_transform(acc9_kmeans_scaled), columns=['c1','c2', 'c3', 'c4', 'c5', 'c6'])

In [ ]:
kmeans = KMeans(n_clusters=7)
kmeans.fit(acc9_pca)
labels = kmeans.predict(acc9_pca)

unique_elements, counts_elements = np.unique(labels, return_counts=True)
clusters = np.asarray((unique_elements, counts_elements))

outliers_fraction = 0.008
distance = getDistanceByPoint(acc9_pca, kmeans)

number_of_outliers = int(outliers_fraction*len(distance))
threshold = distance.nlargest(number_of_outliers).min()
acc9_pca['anomaly_kmeans'] = (distance >= threshold).astype(int)

In [ ]:
acc9_pca.index = acc9.index
acc9_pca.anomaly_kmeans.value_counts()

In [ ]:
acc9_kmeans_scaled['anomaly_kmeans'] = acc9_pca.anomaly_kmeans
bank_main['km_9'] = 0
bank_main.loc[acc9_kmeans_scaled.index,'km_9'] = acc9_kmeans_scaled['anomaly_kmeans']
bank_main['km_9'].value_counts()

In [ ]:
labels = acc9_kmeans_scaled.anomaly_kmeans
colors = {0:'blue', 1:'red'}

fig = plt.figure(1, figsize=(8,8))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(acc9_kmeans_scaled.iloc[:,3], acc9_kmeans_scaled.iloc[:,4], acc9_kmeans_scaled.iloc[:,5], c=labels.apply(lambda x:colors[x]))
ax.set_title(f'KMEANS Anomalies account_no = {accounts[8]} (outliers_fraction = {100*outliers_fraction})%', fontsize=14)
ax.set_xlabel('withdrawal_amt')
ax.set_ylabel('deposit_amt')
ax.set_zlabel('balance_amt')

In [ ]:
acc9_kmeans['anomaly_kmeans'] = acc9_kmeans_scaled['anomaly_kmeans']
df = acc9_kmeans.copy()
a= df.loc[df['anomaly_kmeans']==1, ['date', 'balance_amt']]
plt.figure(figsize=(20,8))
plt.plot(df['date'], df['balance_amt'], color='blue', label='Normal')
plt.scatter(a['date'], a['balance_amt'], color='red', label='Anomaly')
plt.title(f'{accounts[8]} Kmeans Distribution Anomalies')
plt.xlabel('Date')
plt.ylabel('balance_amt')
plt.legend()
plt.show()

#### &emsp;(X) Customer account_no_409000405747

In [ ]:
acc10_kmeans = acc10.copy() 
scaler = StandardScaler()
acc10_kmeans_scaled = pd.DataFrame(scaler.fit_transform(acc10_kmeans[acc10.columns[2:]]), columns=acc10.columns[2:])
acc10_kmeans_scaled.set_index(acc10_kmeans.index, drop=True, inplace=True)
acc10_kmeans_scaled.head(3)

In [ ]:
# Instantiate the clustering model and visualizer
visualizer = KElbowVisualizer(KMeans(), k=(2,15))
plt.figure(figsize=(8,4)).patch.set_facecolor('xkcd:white')
visualizer.fit(acc10_kmeans_scaled)       # Fit the data to the visualizer
# visualizer.show()        # Finalize and render the figure

In [ ]:
kmeans = KMeans(n_clusters=6)
kmeans.fit(acc10_kmeans_scaled)
labels = kmeans.labels_

In [ ]:
fig = plt.figure(1, figsize=(6,6))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(acc10_kmeans_scaled.iloc[:,3], acc10_kmeans_scaled.iloc[:,4], acc10_kmeans_scaled.iloc[:,5], c=labels.astype(np.float), cmap='rainbow')
ax.set_title(f'K MEANS {accounts[9]}', fontsize=14)
ax.set_xlabel('withdrawal_amt')
ax.set_ylabel('deposit_amt')
ax.set_zlabel('balance_amt')

In [ ]:
x = acc10_kmeans_scaled
pca = PCA()
pipeline = make_pipeline(pca)
pipeline.fit(x)
# Plot the principal components against their inertia
features = range(pca.n_components_)
_ = plt.figure(figsize=(15, 5))
_ = plt.bar(features, pca.explained_variance_)
_ = plt.xlabel('PCA feature')
_ = plt.ylabel('Variance')
_ = plt.xticks(features)
_ = plt.title(f"account_no{accounts[9]} Importance of the Principal Components based on inertia")
plt.show()

In [ ]:
acc10_pca = pd.DataFrame(PCA(n_components=3).fit_transform(acc10_kmeans_scaled), columns=['c1','c2', 'c3'])

In [ ]:
kmeans = KMeans(n_clusters=4)
kmeans.fit(acc10_pca)
labels = kmeans.predict(acc10_pca)

unique_elements, counts_elements = np.unique(labels, return_counts=True)
clusters = np.asarray((unique_elements, counts_elements))

outliers_fraction = 0.1
distance = getDistanceByPoint(acc10_pca, kmeans)

number_of_outliers = int(outliers_fraction*len(distance))
threshold = distance.nlargest(number_of_outliers).min()
acc10_pca['anomaly_kmeans'] = (distance >= threshold).astype(int)

In [ ]:
acc10_pca.index = acc10.index
acc10_pca.anomaly_kmeans.value_counts()

In [ ]:
acc10_kmeans_scaled['anomaly_kmeans'] = acc10_pca.anomaly_kmeans
bank_main['km_10'] = 0
bank_main.loc[acc10_kmeans_scaled.index,'km_10'] = acc10_kmeans_scaled['anomaly_kmeans']
bank_main['km_10'].value_counts()

In [ ]:
labels = acc10_kmeans_scaled.anomaly_kmeans
colors = {0:'blue', 1:'red'}

fig = plt.figure(1, figsize=(8,8))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(acc10_kmeans_scaled.iloc[:,3], acc10_kmeans_scaled.iloc[:,4], acc10_kmeans_scaled.iloc[:,5], c=labels.apply(lambda x:colors[x]))
ax.set_title(f'KMEANS Anomalies account_no = {accounts[9]} (outliers_fraction = {100*outliers_fraction})%', fontsize=14)
ax.set_xlabel('withdrawal_amt')
ax.set_ylabel('deposit_amt')
ax.set_zlabel('balance_amt')

In [ ]:
acc10_kmeans['anomaly_kmeans'] = acc10_kmeans_scaled['anomaly_kmeans']
df = acc10_kmeans.copy()
a= df.loc[df['anomaly_kmeans']==1, ['date', 'balance_amt']]
plt.figure(figsize=(20,8))
plt.plot(df['date'], df['balance_amt'], color='blue', label='Normal')
plt.scatter(a['date'], a['balance_amt'], color='red', label='Anomaly')
plt.title(f'{accounts[9]} Kmeans Distribution Anomalies')
plt.xlabel('Date')
plt.ylabel('balance_amt')
plt.legend()
plt.show()

In [ ]:
bank_main['accounts_kmeans'] = bank_main.km_1 + bank_main.km_2 + bank_main.km_3 + bank_main.km_4 + bank_main.km_5 + bank_main.km_6 + bank_main.km_7 + bank_main.km_8 + bank_main.km_9 + bank_main.km_10

In [ ]:
bank_main['accounts_kmeans'].value_counts()

In [ ]:
bank_main['anomaly_sum_2'] = bank_main.accounts_isolation_forest + bank_main.accounts_SVM + bank_main.accounts_gaussian_distribution + bank_main.accounts_kmeans

## 5. Grading Anomalies

In [ ]:
bank_graded = bank_main.copy()

In [ ]:
drop_lst =['iso_1', 'iso_2', 'iso_3', 'iso_4', 'iso_5', 'iso_6', 'iso_7', 'iso_8', 'iso_9', 'iso_10',\
 'svm_1', 'svm_2', 'svm_3', 'svm_4', 'svm_5', 'svm_6', 'svm_7', 'svm_8', 'svm_9', 'svm_10',\
 'gauss_10', 'gauss_2', 'gauss_1', 'gauss_3', 'gauss_4', 'gauss_5', 'gauss_6', 'gauss_7', 'gauss_8', 'gauss_9',\
 'km_1', 'km_2', 'km_3', 'km_4', 'km_5', 'km_6', 'km_7', 'km_8', 'km_9', 'km_10']

bank_graded.drop(columns=drop_lst, axis=1, inplace=True)

In [ ]:
bank_graded['weighted_anomaly'] = bank_graded.anomaly_sum_1 + 2 * bank_graded.anomaly_sum_2
bank_graded['weighted_anomaly'].value_counts()

In [ ]:
bank_graded['ranks'] =  0

In [ ]:
bank_graded.loc[bank_graded.weighted_anomaly>8, 'ranks'] = 1
bank_graded.loc[(bank_graded.weighted_anomaly>4) & (bank_graded.weighted_anomaly<=8),'ranks'] = 2
bank_graded.loc[(bank_graded.weighted_anomaly>2) & (bank_graded.weighted_anomaly<=4), 'ranks'] = 3
bank_graded.ranks.value_counts()

Since we have essentially determined **weighted_anomaly** using machine learning algorithms, we now want to check some suspicious points that we discovered during the EDA phase. And finally, we synthesize a rank value for each instance.

### day_diff vs. weighted_anomaly

In [ ]:
bank_graded[bank_graded.day_diff<0][['day_diff', 'weighted_anomaly', 'ranks']]

In [ ]:
bank_graded.loc[bank_graded.day_diff<0, 'ranks'] = 1

In [ ]:
bank_graded[bank_graded.day_diff>4][['day_diff', 'weighted_anomaly', 'ranks']]

In [ ]:
bank_graded.loc[(bank_graded.day_diff>4) & (bank_graded.day_diff<6) & (bank_graded.ranks==0), 'ranks'] = 3
bank_graded.loc[bank_graded.day_diff>10, 'ranks'] = 1

In [ ]:
bank_graded[bank_graded.day_diff>4].ranks.value_counts()

### details_null_penalty vs. weighted_anomaly

In [ ]:
bank_graded[bank_graded.details_null_penalty>0][['details_null_penalty', 'weighted_anomaly', 'ranks']]

In [ ]:
bank_graded[bank_graded.details_null_penalty>0].ranks.value_counts()

### weekend_date vs. weighted_anomaly

In [ ]:
bank_graded[bank_graded.weekend_date>0][['weekend_date', 'weekend_value_date', 'weighted_anomaly', 'ranks']]

In [ ]:
bank_graded[bank_graded.weekend_date>0].ranks.value_counts()

In [ ]:
bank_graded[bank_graded.weekend_date + bank_graded.weekend_value_date==1][['weekend_date', 'weekend_value_date', 'weighted_anomaly', 'ranks']]

In [ ]:
bank_graded.loc[(bank_graded.weekend_date + bank_graded.weekend_value_date==1) & (bank_graded.weekend_date ==1) & (bank_graded.ranks==0), 'ranks'] = 3

In [ ]:
bank_graded[bank_graded.weekend_date + bank_graded.weekend_value_date==1].ranks.value_counts()

### cheque_penalty vs. weighted_anomaly

In [ ]:
bank_graded[bank_graded.cheque_penalty>0][['cheque_penalty', 'weighted_anomaly', 'ranks']]

In [ ]:
bank_graded[bank_graded.cheque_penalty>0].ranks.value_counts()

In [ ]:
bank_graded.ranks.value_counts()

## 6. Conclusion

In [ ]:
bank_ranked = bank_graded.copy()

In [ ]:
drop_lst = ['account_no_1196428', 'account_no_1196711', 'account_no_409000362497', 'account_no_409000405747', 'account_no_409000425051',\
            'account_no_409000438611', 'account_no_409000438620', 'account_no_409000493201', 'account_no_409000493210', 'account_no_409000611074',\
            'anomaly_kmeans', 'anomaly_isolation_forest', 'anomaly_one_class_svm', 'anomaly_gaussian_elliptic_envelope', 'anomaly_sum_1',\
            'accounts_isolation_forest', 'accounts_SVM', 'accounts_gaussian_distribution', 'accounts_kmeans', 'anomaly_sum_2']

bank_ranked.drop(columns=drop_lst, axis=1, inplace=True)

In [ ]:
bank_ranked['account_no'] = bank_premodel.account_no

In [ ]:
bank_ranked.head()

In [ ]:
bank_ranked.to_excel('bank_transaction_ranked.xlsx')

In [ ]:
bank_ranked[bank_ranked.deposit_over_balance < 0].ranks.value_counts()

In [ ]:
bank_ranked[bank_ranked.ranks > 0]